In [1]:
# !pip install fuzzywuzzy
# !pip install python-Levenshtein
# !pip install rapidfuzz
# !pip install jaro-winkler

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display
from time import gmtime, strftime
import sys
import os
import io

import string
import re
# import itertools
# import nltk
# nltk.download('stopwords')

from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from rapidfuzz import fuzz as rfuzz
import jaro

In [74]:
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_rows', 50)

In [3]:
def frequency_ct(ngram_list):
    freq_dict = {}
    for ngram in ngram_list:
        if ngram not in freq_dict:
            freq_dict[ngram] = 0
        freq_dict[ngram] +=1
    return freq_dict

In [4]:
df = pd.read_csv('data/parsed_bahamas_addresses.csv')

In [5]:
df.shape

(2258, 9)

In [6]:
df.head()

node_id  \
0  24000001   
1  24000002   
2  24000003   
3  24000004   
4  24000005   

                                                                                              address  \
0                                     ANNEX FREDERICK & SHIRLEY STS, P.O. BOX N-4805, NASSAU, BAHAMAS   
1                                    SUITE E-2,UNION COURT BUILDING, P.O. BOX N-8188, NASSAU, BAHAMAS   
2                                      LYFORD CAY HOUSE, LYFORD CAY, P.O. BOX N-7785, NASSAU, BAHAMAS   
3  P.O. BOX N-3708 BAHAMAS FINANCIAL CENTRE, P.O. BOX N-3708 SHIRLEY & CHARLOTTE STS, NASSAU, BAHAMAS   
4                           LYFORD CAY HOUSE, 3RD FLOOR, LYFORD CAY, P.O. BOX N-3024, NASSAU, BAHAMAS   

  name countries country_codes       sourceID  \
0  NaN   Bahamas           BHS  Bahamas Leaks   
1  NaN   Bahamas           BHS  Bahamas Leaks   
2  NaN   Bahamas           BHS  Bahamas Leaks   
3  NaN   Bahamas           BHS  Bahamas Leaks   
4  NaN   Bahamas           BHS  Bahamas Leaks   

                                             valid_until note  \
0  The Bahamas Leaks data is current through early 2016.  NaN   
1  The Bahamas Leaks data is current through early 2016.  NaN   
2  The Bahamas Leaks data is current through early 2016.  NaN   
3  The Bahamas Leaks data is current through early 2016.  NaN   
4  The Bahamas Leaks data is current through early 2016.  NaN   

                                                                                  working_address  
0                                  annex frederick and shirley street po box n4805 nassau bahamas  
1                                       suite e2 union court building po box n8188 nassau bahamas  
2                                         lyford cay house lyford cay po box n7785 nassau bahamas  
3  po box n3708 bahamas financial centre po box n3708 shirley and charlotte street nassau bahamas  
4                             lyford cay house third floor lyford cay po box n3024 nassau bahamas

In [7]:
df['address_wordlist'] = df['working_address'].fillna('').str.split()

In [8]:
freq_df = pd.DataFrame.from_dict(
    frequency_ct(df['address_wordlist'].sum()
                ), orient='index').reset_index().rename(
    columns={'index':'word', 0:'count'}).sort_values('count', ascending=False)

In [9]:
freq_df.shape

(2040, 2)

In [10]:
freq_df.head(60)

word  count
9          bahamas   2321
8           nassau   2040
6              box   1484
5               po   1430
4           street   1128
2              and    627
3          shirley    489
10           suite    447
34             bay    431
14        building    329
18           house    318
22          centre    299
13           court    290
60          avenue    280
23       charlotte    255
12           union    234
162      elizabeth    228
25           floor    219
638              2    215
419              e    203
111           bank    189
57             the    187
33            east    186
112           lane    176
21       financial    163
49               n    161
42            west    159
109         square    156
919        limited    145
108        saffrey    143
35      providence    129
15           n8188    117
374            new    116
466            205    112
186       freeport    107
16          lyford     89
53            road     86
188         bahama     84
940              p     82
439             co     75
17             cay     75
199          grand     74
228             of     73
99          second     71
24           third     69
176          trust     68
55          fourth     67
72           place     65
73     marlborough     61
445            ltd     61
47       corporate     57
137          drive     56
957           8188     54
74           queen     52
1088             o     51
81              cb     49
240         island     45
978        company     44
71    winterbotham     44
37            hill     41

## Fuzzy matching

In [11]:
df.head()

node_id  \
0  24000001   
1  24000002   
2  24000003   
3  24000004   
4  24000005   

                                                                                              address  \
0                                     ANNEX FREDERICK & SHIRLEY STS, P.O. BOX N-4805, NASSAU, BAHAMAS   
1                                    SUITE E-2,UNION COURT BUILDING, P.O. BOX N-8188, NASSAU, BAHAMAS   
2                                      LYFORD CAY HOUSE, LYFORD CAY, P.O. BOX N-7785, NASSAU, BAHAMAS   
3  P.O. BOX N-3708 BAHAMAS FINANCIAL CENTRE, P.O. BOX N-3708 SHIRLEY & CHARLOTTE STS, NASSAU, BAHAMAS   
4                           LYFORD CAY HOUSE, 3RD FLOOR, LYFORD CAY, P.O. BOX N-3024, NASSAU, BAHAMAS   

  name countries country_codes       sourceID  \
0  NaN   Bahamas           BHS  Bahamas Leaks   
1  NaN   Bahamas           BHS  Bahamas Leaks   
2  NaN   Bahamas           BHS  Bahamas Leaks   
3  NaN   Bahamas           BHS  Bahamas Leaks   
4  NaN   Bahamas           BHS  Bahamas Leaks   

                                             valid_until note  \
0  The Bahamas Leaks data is current through early 2016.  NaN   
1  The Bahamas Leaks data is current through early 2016.  NaN   
2  The Bahamas Leaks data is current through early 2016.  NaN   
3  The Bahamas Leaks data is current through early 2016.  NaN   
4  The Bahamas Leaks data is current through early 2016.  NaN   

                                                                                  working_address  \
0                                  annex frederick and shirley street po box n4805 nassau bahamas   
1                                       suite e2 union court building po box n8188 nassau bahamas   
2                                         lyford cay house lyford cay po box n7785 nassau bahamas   
3  po box n3708 bahamas financial centre po box n3708 shirley and charlotte street nassau bahamas   
4                             lyford cay house third floor lyford cay po box n3024 nassau bahamas   

                                                                                                 address_wordlist  
0                                       [annex, frederick, and, shirley, street, po, box, n4805, nassau, bahamas]  
1                                            [suite, e2, union, court, building, po, box, n8188, nassau, bahamas]  
2                                              [lyford, cay, house, lyford, cay, po, box, n7785, nassau, bahamas]  
3  [po, box, n3708, bahamas, financial, centre, po, box, n3708, shirley, and, charlotte, street, nassau, bahamas]  
4                                [lyford, cay, house, third, floor, lyford, cay, po, box, n3024, nassau, bahamas]

In [12]:
print('Sample 1:', df['working_address'][0])
print('Sample 2:', df['working_address'][1])

Sample 1: annex frederick and shirley street po box n4805 nassau bahamas
Sample 2: suite e2 union court building po box n8188 nassau bahamas


### fuzzywuzzy implimentation

In [13]:
print('Ratio:', fuzz.ratio(df['working_address'][0], df['working_address'][1]))
print('Partial ratio:', fuzz.partial_ratio(df['working_address'][0], df['working_address'][1]))
print('Token sort ratio:', fuzz.token_sort_ratio(df['working_address'][0], df['working_address'][1]))
print('Token set ratio:', fuzz.token_set_ratio(df['working_address'][0], df['working_address'][1]))

Ratio: 57
Partial ratio: 58
Token sort ratio: 55
Token set ratio: 59


In [14]:
print('Ratio:', fuzz.ratio(df['working_address'][1], df['working_address'][0]))
print('Partial ratio:', fuzz.partial_ratio(df['working_address'][1], df['working_address'][0]))
print('Token sort ratio:', fuzz.token_sort_ratio(df['working_address'][1], df['working_address'][0]))
print('Token set ratio:', fuzz.token_set_ratio(df['working_address'][1], df['working_address'][0]))

Ratio: 57
Partial ratio: 58
Token sort ratio: 55
Token set ratio: 59


### rapidfuzz implimentation

In [15]:
print('Ratio:', rfuzz.ratio(df['working_address'][0], df['working_address'][1]))
print('Partial ratio:', rfuzz.partial_ratio(df['working_address'][0], df['working_address'][1]))
print('Token sort ratio:', rfuzz.token_sort_ratio(df['working_address'][0], df['working_address'][1]))
print('Token set ratio:', rfuzz.token_set_ratio(df['working_address'][0], df['working_address'][1]))

Ratio: 57.14285714285714
Partial ratio: 65.93406593406593
Token sort ratio: 55.46218487394958
Token set ratio: 58.8235294117647


In [16]:
print('Ratio:', rfuzz.ratio(df['working_address'][1], df['working_address'][0]))
print('Partial ratio:', rfuzz.partial_ratio(df['working_address'][1], df['working_address'][0]))
print('Token sort ratio:', rfuzz.token_sort_ratio(df['working_address'][1], df['working_address'][0]))
print('Token set ratio:', rfuzz.token_set_ratio(df['working_address'][1], df['working_address'][0]))

Ratio: 57.14285714285714
Partial ratio: 65.93406593406593
Token sort ratio: 55.46218487394958
Token set ratio: 58.8235294117647


### Jaro Winkler

In [17]:
print('Jaro winkler 1:', jaro.jaro_winkler_metric(df['working_address'][0], df['working_address'][1]))
print('Jaro winkler 2:', jaro.jaro_winkler_metric(df['working_address'][1], df['working_address'][0]))

Jaro winkler 1: 0.6764432336154215
Jaro winkler 2: 0.6764432336154215


In [18]:
goodmans_series = df.loc[df['working_address'].str.contains('bay corporate'), 'working_address']
goodmans_series

9                                               ground floor goodmans bay corporate ce po box n 3933 nassau bahamas
63                                               goodmans bay corporate centre west bay po box n3015 nassau bahamas
100                                     goodmans bay corporate centre po box cb10976 west bay street nassau bahamas
116                                           goodmans bay corporate centre suite 261 po box cb12762 nassau bahamas
248                            goodmans bay corporate centre po box ss5498 suite 261 west bay street nassau bahamas
268                                                     goodmans bay corporate centre po box cb12407 nassau bahamas
548                              second floor goodmans bay corporate centre suite 261 po box cb12762 nassau bahamas
756                                      goodman0s bay corporate centre west bay street po box n4938 nassau bahamas
758                                       goodmans bay corporate center 

In [19]:
goodmans_series.shape

(37,)

In [20]:
goodmans_series.iloc[0]

'ground floor goodmans bay corporate ce po box n 3933 nassau bahamas'

### process.extract

In [20]:
process.extract(goodmans_series.iloc[0], goodmans_series, limit=10)

[('ground floor goodmans bay corporate ce po box n 3933 nassau bahamas',
  100,
  9),
 ('ground floor goodmans bay corporate ce po box n 3933 nassau bahamas',
  100,
  1975),
 ('second floor goodmans bay corporate centre', 86, 1197),
 ('co cotswold group goodmans bay corporate centre second floor po box cb 12762 suite 261 nassau bahamas',
  86,
  1800),
 ('cibc trust company bahamas limited first floor goodmans bay corporate centre west bay street nassau bahamas',
  86,
  2133),
 ('goodmans bay corporate centre po box cb12407 nassau bahamas', 81, 268),
 ('second  floor goodmans bay corporate centre suite 261 po box cb12762 nassau bahamas',
  81,
  548),
 ('goodmans bay corporate centre po box cb10976 nassau bahamas', 81, 2068),
 ('goodmans bay corporate centre west bay po box n3015 nassau bahamas',
  79,
  63),
 ('goodmans bay corporate centre west bay po box n3015 nassau bahamas',
  79,
  2120)]

In [21]:
for scorer in [fuzz.ratio, fuzz.partial_ratio, fuzz.token_sort_ratio, fuzz.token_set_ratio, jaro.jaro_winkler_metric]:
    print(str(scorer))
    display(process.extract(goodmans_series.iloc[0], goodmans_series, scorer=scorer, limit=40))
    print('\n')

<function ratio at 0x7fbebd75d5e0>


[('ground floor goodmans bay corporate ce po box n 3933 nassau bahamas',
  100,
  9),
 ('ground floor goodmans bay corporate ce po box n 3933 nassau bahamas',
  100,
  1975),
 ('goodmans bay corporate centre po box cb10976 nassau bahamas', 78, 2068),
 ('second  floor goodmans bay corporate centre suite 261 po box cb12762 nassau bahamas',
  77,
  548),
 ('goodmans bay corporate centre po box cb12407 nassau bahamas', 76, 268),
 ('goodmans bay corporate centre west bay street po box n3933 nassau bahamas',
  76,
  1414),
 ('goodmans bay corporate centre west bay street po box n3933 nassau bahamas',
  76,
  1511),
 ('goodmans bay corporate centre west bay street po box n3933 nassau bahamas',
  76,
  1610),
 ('first floor goodmans bay corporate centre bay street nassau bahamas',
  76,
  1707),
 ('goodmans bay corporate centre west bay po box n3015 nassau bahamas',
  75,
  63),
 ('goodmans bay corporate centre west bay po box n3015 nassau bahamas',
  75,
  2120),
 ('goodmans bay corporate cen



<function partial_ratio at 0x7fbebd75d820>


[('ground floor goodmans bay corporate ce po box n 3933 nassau bahamas',
  100,
  9),
 ('ground floor goodmans bay corporate ce po box n 3933 nassau bahamas',
  100,
  1975),
 ('goodmans bay corporate centre po box cb10976 nassau bahamas', 87, 2068),
 ('goodmans bay corporate centre po box cb12407 nassau bahamas', 85, 268),
 ('goodmans bay corporate centre west bay po box n3015 nassau bahamas',
  83,
  63),
 ('second floor goodmans bay corporate centre', 83, 1197),
 ('goodmans bay corporate centre west bay po box n3015 nassau bahamas',
  83,
  2120),
 ('goodmans bay corporate centre west bay street  nassau  bahamas', 78, 765),
 ('first floor goodmans bay corporate centre bay street nassau bahamas',
  77,
  1707),
 ('cibc trust company bahamas limited first floor goodmans bay corporate centre west bay street nassau bahamas',
  76,
  2133),
 ('goodmans bay corporate centre west bay street nassau the bahamas', 75, 766),
 ('co third fl goodmans bay corporate centre west bay street nassau b



<function token_sort_ratio at 0x7fbebd75da60>


[('ground floor goodmans bay corporate ce po box n 3933 nassau bahamas',
  100,
  9),
 ('ground floor goodmans bay corporate ce po box n 3933 nassau bahamas',
  100,
  1975),
 ('goodmans bay corporate centre po box cb12407 nassau bahamas', 75, 268),
 ('goodmans bay corporate centre po box cb10976 nassau bahamas', 75, 2068),
 ('goodmans bay corporate centre west bay po box n3015 nassau bahamas',
  74,
  63),
 ('goodmans bay corporate centre west bay po box n3015 nassau bahamas',
  74,
  2120),
 ('first floor goodmans bay corporate centre bay street nassau bahamas',
  73,
  1707),
 ('second  floor goodmans bay corporate centre suite 261 po box cb12762 nassau bahamas',
  72,
  548),
 ('goodmans bay corporate centre suite 261 po box cb12762 nassau bahamas',
  71,
  116),
 ('goodmans bay corporate centre third floor west bay street nassau bahamas',
  71,
  762),
 ('goodmans bay corporate centre first floor west bay street nassau bahamas',
  71,
  1667),
 ('goodman0s bay corporate centre wes



<function token_set_ratio at 0x7fbebd75dca0>


[('ground floor goodmans bay corporate ce po box n 3933 nassau bahamas',
  100,
  9),
 ('ground floor goodmans bay corporate ce po box n 3933 nassau bahamas',
  100,
  1975),
 ('goodmans bay corporate centre po box cb12407 nassau bahamas', 85, 268),
 ('second  floor goodmans bay corporate centre suite 261 po box cb12762 nassau bahamas',
  85,
  548),
 ('co cotswold group goodmans bay corporate centre second floor po box cb 12762 suite 261 nassau bahamas',
  85,
  1800),
 ('goodmans bay corporate centre po box cb10976 nassau bahamas', 85, 2068),
 ('goodmans bay corporate centre west bay po box n3015 nassau bahamas',
  83,
  63),
 ('goodmans bay corporate centre west bay po box n3015 nassau bahamas',
  83,
  2120),
 ('first floor goodmans bay corporate centre bay street nassau bahamas',
  81,
  1707),
 ('goodmans bay corporate centre west bay street  nassau  bahamas', 80, 765),
 ('second floor goodmans bay corporate centre', 80, 1197),
 ('goodmans bay corporate centre po box cb10976 west



<function jaro_winkler_metric at 0x7fbebd767310>


[('ground floor goodmans bay corporate ce po box n 3933 nassau bahamas',
  1.0,
  9),
 ('ground floor goodmans bay corporate ce po box n 3933 nassau bahamas',
  1.0,
  1975),
 ('goodmans bay corporate centre po box cb12407 nassau bahamas',
  0.7761178851505186,
  268),
 ('goodmans bay corporate centre po box cb10976 nassau bahamas',
  0.775915536971176,
  2068),
 ('goodmans bay corporate centre west bay street po box n3933 nassau bahamas',
  0.774441468071761,
  1414),
 ('goodmans bay corporate centre west bay street po box n3933 nassau bahamas',
  0.774441468071761,
  1511),
 ('goodmans bay corporate centre west bay street po box n3933 nassau bahamas',
  0.774441468071761,
  1610),
 ('second floor goodmans bay corporate centre', 0.7660186839291317, 1197),
 ('goodmans bay corporate center west bay street po box n4938 nassau bahamas',
  0.7631546664657964,
  1524),
 ('goodman0s bay corporate centre west bay street po box n4938 nassau bahamas',
  0.7599891722044119,
  756),
 ('goodmans b

In [22]:
for scorer in [rfuzz.ratio, rfuzz.partial_ratio, rfuzz.token_sort_ratio, rfuzz.token_set_ratio, jaro.jaro_winkler_metric]:
    print(str(scorer))
    display(process.extract(goodmans_series.iloc[0], goodmans_series, scorer=scorer, limit=40))
    print('\n')

<cyfunction ratio at 0x7fbebd70e860>


[('ground floor goodmans bay corporate ce po box n 3933 nassau bahamas',
  100.0,
  9),
 ('ground floor goodmans bay corporate ce po box n 3933 nassau bahamas',
  100.0,
  1975),
 ('goodmans bay corporate centre po box cb10976 nassau bahamas',
  77.77777777777779,
  2068),
 ('second  floor goodmans bay corporate centre suite 261 po box cb12762 nassau bahamas',
  77.33333333333333,
  548),
 ('goodmans bay corporate centre po box cb12407 nassau bahamas',
  76.19047619047619,
  268),
 ('first floor goodmans bay corporate centre bay street nassau bahamas',
  76.11940298507463,
  1707),
 ('goodmans bay corporate centre west bay street po box n3933 nassau bahamas',
  75.71428571428571,
  1414),
 ('goodmans bay corporate centre west bay street po box n3933 nassau bahamas',
  75.71428571428571,
  1511),
 ('goodmans bay corporate centre west bay street po box n3933 nassau bahamas',
  75.71428571428571,
  1610),
 ('goodmans bay corporate centre west bay po box n3015 nassau bahamas',
  75.1879699



<cyfunction partial_ratio at 0x7fbebd70e930>


[('ground floor goodmans bay corporate ce po box n 3933 nassau bahamas',
  100.0,
  9),
 ('ground floor goodmans bay corporate ce po box n 3933 nassau bahamas',
  100.0,
  1975),
 ('second floor goodmans bay corporate centre', 87.5, 1197),
 ('goodmans bay corporate centre po box cb10976 nassau bahamas',
  86.72566371681415,
  2068),
 ('goodmans bay corporate centre po box cb12407 nassau bahamas',
  84.95575221238938,
  268),
 ('goodmans bay corporate centre west bay po box n3015 nassau bahamas',
  83.33333333333334,
  63),
 ('goodmans bay corporate centre west bay po box n3015 nassau bahamas',
  83.33333333333334,
  2120),
 ('first floor goodmans bay corporate centre bay street nassau bahamas',
  78.125,
  1707),
 ('goodmans bay corporate centre west bay street  nassau  bahamas',
  77.58620689655173,
  765),
 ('cibc trust company bahamas limited first floor goodmans bay corporate centre west bay street nassau bahamas',
  76.11940298507463,
  2133),
 ('co third fl goodmans bay corporate



<cyfunction token_sort_ratio at 0x7fbebd70ead0>


[('ground floor goodmans bay corporate ce po box n 3933 nassau bahamas',
  100.0,
  9),
 ('ground floor goodmans bay corporate ce po box n 3933 nassau bahamas',
  100.0,
  1975),
 ('goodmans bay corporate centre po box cb12407 nassau bahamas',
  74.60317460317461,
  268),
 ('goodmans bay corporate centre po box cb10976 nassau bahamas',
  74.60317460317461,
  2068),
 ('goodmans bay corporate centre west bay po box n3015 nassau bahamas',
  73.6842105263158,
  63),
 ('goodmans bay corporate centre west bay po box n3015 nassau bahamas',
  73.6842105263158,
  2120),
 ('first floor goodmans bay corporate centre bay street nassau bahamas',
  73.13432835820896,
  1707),
 ('second  floor goodmans bay corporate centre suite 261 po box cb12762 nassau bahamas',
  72.48322147651007,
  548),
 ('goodmans bay corporate centre suite 261 po box cb12762 nassau bahamas',
  70.58823529411764,
  116),
 ('goodmans bay corporate centre third floor west bay street nassau bahamas',
  70.50359712230217,
  762),




<cyfunction token_set_ratio at 0x7fbebd70eba0>


[('ground floor goodmans bay corporate ce po box n 3933 nassau bahamas',
  100.0,
  9),
 ('ground floor goodmans bay corporate ce po box n 3933 nassau bahamas',
  100.0,
  1975),
 ('second  floor goodmans bay corporate centre suite 261 po box cb12762 nassau bahamas',
  85.47008547008546,
  548),
 ('co cotswold group goodmans bay corporate centre second floor po box cb 12762 suite 261 nassau bahamas',
  85.47008547008546,
  1800),
 ('goodmans bay corporate centre po box cb12407 nassau bahamas',
  85.4368932038835,
  268),
 ('goodmans bay corporate centre po box cb10976 nassau bahamas',
  85.4368932038835,
  2068),
 ('goodmans bay corporate centre west bay po box n3015 nassau bahamas',
  83.01886792452831,
  63),
 ('goodmans bay corporate centre west bay po box n3015 nassau bahamas',
  83.01886792452831,
  2120),
 ('first floor goodmans bay corporate centre bay street nassau bahamas',
  81.13207547169812,
  1707),
 ('second floor goodmans bay corporate centre', 80.0, 1197),
 ('goodmans b



<function jaro_winkler_metric at 0x7fbebd767310>


[('ground floor goodmans bay corporate ce po box n 3933 nassau bahamas',
  1.0,
  9),
 ('ground floor goodmans bay corporate ce po box n 3933 nassau bahamas',
  1.0,
  1975),
 ('goodmans bay corporate centre po box cb12407 nassau bahamas',
  0.7761178851505186,
  268),
 ('goodmans bay corporate centre po box cb10976 nassau bahamas',
  0.775915536971176,
  2068),
 ('goodmans bay corporate centre west bay street po box n3933 nassau bahamas',
  0.774441468071761,
  1414),
 ('goodmans bay corporate centre west bay street po box n3933 nassau bahamas',
  0.774441468071761,
  1511),
 ('goodmans bay corporate centre west bay street po box n3933 nassau bahamas',
  0.774441468071761,
  1610),
 ('second floor goodmans bay corporate centre', 0.7660186839291317, 1197),
 ('goodmans bay corporate center west bay street po box n4938 nassau bahamas',
  0.7631546664657964,
  1524),
 ('goodman0s bay corporate centre west bay street po box n4938 nassau bahamas',
  0.7599891722044119,
  756),
 ('goodmans b

## Solutioning

I'm trying to use fuzzy matching to identify and resolve duplicates. Having not worked with this before, I want to see the match values so that I can determine an appropriate threshold. To do this, I need to come up with a way to process the values, store the data, and analyize it.

### Metrics

The Goodman's Bay Corporate Centre example above returned the following ranges for each metric which make me think that some metrics are better for certain use cases than others:

**Fuzzywuzzy**

- Ratio: 100 - 58
- Partial ratio: 100 - 51
- Token sort: 100 - 60
- Token set: 100 - 66
- Jaro-Winkler: 1.0 - 0.65

**Rapidfuzz**

- Ratio: 100 - 57.8
- Partial ratio: 100 - 58.6
- Token sort: 100 - 59.8
- Token set: 100 - 65.7
- Jaro-Winkler: 1.0 - 0.65

### Storage format

From the Goodman's Bay Corporate Centre results, the relevant information I think I'll need includes:

<table>
    <tr>
        <td>address_index</td>
        <td>address</td>
        <td>match_index</td>
        <td>match</td>
        <td>ratio_score</td>
        <td>partial_ratio_score</td>
        <td>token_sort_score</td>
        <td>token_set_score</td>
        <td>jaro_winkler_score</td>
    </tr>
    <tr>
        <td>9</td>
        <td>'ground floor goodmans bay corporate ce po box n 3933 nassau bahamas'</td>
        <td>1975</td>
        <td>'ground floor goodmans bay corporate ce po box n 3933 nassau bahamas'</td>
        <td>100</td>
        <td>100</td>
        <td>100</td>
        <td>100</td>
        <td>1.0</td>
    </tr>
    <tr>
        <td>9</td>
        <td>'ground floor goodmans bay corporate ce po box n 3933 nassau bahamas'</td>
        <td>2068</td>
        <td>'goodmans bay corporate centre po box cb10976 nassau bahamas'</td>
        <td>78</td>
        <td>87</td>
        <td>75</td>
        <td>85</td>
        <td>0.78</td>
    </tr>
    <tr>
        <td>9</td>
        <td>'ground floor goodmans bay corporate ce po box n 3933 nassau bahamas'</td>
        <td>548</td>
        <td>'second  floor goodmans bay corporate centre suite 261 po box cb12762 nassau bahamas'</td>
        <td>77</td>
        <td>69</td>
        <td>72</td>
        <td>85</td>
        <td>0.74</td>
    </tr>
    <tr>
        <td>9</td>
        <td>'ground floor goodmans bay corporate ce po box n 3933 nassau bahamas'</td>
        <td>268</td>
        <td>'goodmans bay corporate centre po box cb12407 nassau bahamas'</td>
        <td>76</td>
        <td>85</td>
        <td>75</td>
        <td>85</td>
        <td>0.78</td>
    </tr>
    <tr>
        <td>9</td>
        <td>'ground floor goodmans bay corporate ce po box n 3933 nassau bahamas'</td>
        <td>1511</td>
        <td>'goodmans bay corporate centre west bay street po box n3933 nassau bahamas'</td>
        <td>76</td>
        <td>70</td>
        <td>70</td>
        <td>79</td>
        <td>0.77</td>
    </tr>
    <tr>
        <td>9</td>
        <td>'ground floor goodmans bay corporate ce po box n 3933 nassau bahamas'</td>
        <td>1707</td>
        <td>'first floor goodmans bay corporate centre bay street nassau bahamas'</td>
        <td>76</td>
        <td>77</td>
        <td>73</td>
        <td>81</td>
        <td>0.73</td>
    </tr>
</table>

The next question is: how do I get all of this information into a dataframe?

The `process.extract` function I tried earlier limits on the number of results returned, not the quality of results. With the address datasets I'll be working with, there are potentially large groups of related data (especially when I get into countries with large representation in the original dataset). Because of this, I won't be using the `process.extract` function. I'm going to prototype using a smaller, easier dataset: the word frequency dataset.

In [21]:
freq_df.sort_index()

word  count
0         annex      8
1     frederick     30
2           and    627
3       shirley    489
4        street   1128
...         ...    ...
2035   sterline      1
2036        bav      1
2037       hast      1
2038        coj      1
2039       2ntl      1

[2040 rows x 2 columns]

In [24]:
freq_df['word'].sort_index()[:5]

0        annex
1    frederick
2          and
3      shirley
4       street
Name: word, dtype: object

In [25]:
print(f'''Original index: {0}
Original value: {freq_df.iloc[0, 0]}
Match index: {1}
Match value: {freq_df.iloc[1, 0]}
Metric output: {rfuzz.ratio(freq_df.iloc[0, 0], freq_df.iloc[1, 0])}
''')

Original index: 0
Original value: bahamas
Match index: 1
Match value: nassau
Metric output: 30.76923076923077



In [26]:
for o_i, o_v in enumerate(freq_df['word'].sort_index()[:5]):
    for m_i, m_v in enumerate(freq_df['word'].sort_index()[:5]):
        if o_i != m_i:
            print([o_i, o_v, m_i, m_v, rfuzz.ratio(o_v, m_v), rfuzz.partial_ratio(o_v, m_v), rfuzz.token_sort_ratio(o_v, m_v), rfuzz.token_set_ratio(o_v, m_v), jaro.jaro_winkler_metric(o_v, m_v)])
            

[0, 'annex', 1, 'frederick', 14.28571428571429, 25.0, 14.28571428571429, 14.285714285714292, 0.43703703703703706]
[0, 'annex', 2, 'and', 50.0, 80.0, 50.0, 50.0, 0.6888888888888888]
[0, 'annex', 3, 'shirley', 16.666666666666664, 28.57142857142857, 16.666666666666664, 16.66666666666667, 0.44761904761904764]
[0, 'annex', 4, 'street', 18.181818181818176, 28.57142857142857, 18.181818181818176, 18.181818181818187, 0.45555555555555555]
[1, 'frederick', 0, 'annex', 14.28571428571429, 25.0, 14.28571428571429, 14.285714285714292, 0.43703703703703706]
[1, 'frederick', 2, 'and', 16.666666666666664, 33.333333333333336, 16.666666666666664, 16.66666666666667, 0.48148148148148145]
[1, 'frederick', 3, 'shirley', 25.0, 40.0, 25.0, 25.0, 0.5026455026455027]
[1, 'frederick', 4, 'street', 40.0, 54.54545454545454, 40.0, 40.0, 0.611111111111111]
[2, 'and', 0, 'annex', 50.0, 80.0, 50.0, 50.0, 0.6888888888888888]
[2, 'and', 1, 'frederick', 16.666666666666664, 33.333333333333336, 16.666666666666664, 16.66666666

In [93]:
# def calc_fuzz_df(df, column):
#     row_list = []
    
#     for o_i, o_v in enumerate(df[column].sort_index()):
#         for m_i, m_v in enumerate(df[column].sort_index()):
#             if o_i != m_i:
#                 dict1 = {
#                     'original_index': o_i,
#                     'original_value': o_v,
#                     'match_index': m_i,
#                     'match_value': m_v,
#                     'ratio_score': rfuzz.ratio(o_v, m_v),
#                     'partial_ratio_score': rfuzz.partial_ratio(o_v, m_v),
#                     'token_sort_score': rfuzz.token_sort_ratio(o_v, m_v),
#                     'token_set_score': rfuzz.token_set_ratio(o_v, m_v),
#                     'jaro_winkler_score': jaro.jaro_winkler_metric(o_v, m_v)
#                 }
#                 row_list.append(dict1)
#     score_df = pd.DataFrame(row_list)
        
#     return score_df

In [94]:
# fuzzy_words_df = calc_fuzz_df(freq_df, 'word')
# fuzzy_words_df

original_index original_value  match_index match_value  ratio_score  \
0                     0          annex            1   frederick    14.285714   
1                     0          annex            2         and    50.000000   
2                     0          annex            3     shirley    16.666667   
3                     0          annex            4      street    18.181818   
4                     0          annex            5          po     0.000000   
...                 ...            ...          ...         ...          ...   
4370185            2090           2ntl         2085    montagne    33.333333   
4370186            2090           2ntl         2086    sterline    33.333333   
4370187            2090           2ntl         2087         bav     0.000000   
4370188            2090           2ntl         2088        hast    25.000000   
4370189            2090           2ntl         2089        cojp     0.000000   

         partial_ratio_score  token_sort_score  token_set_score  \
0                  25.000000         14.285714        14.285714   
1                  80.000000         50.000000        50.000000   
2                  28.571429         16.666667        16.666667   
3                  28.571429         18.181818        18.181818   
4                   0.000000          0.000000         0.000000   
...                      ...               ...              ...   
4370185            50.000000         33.333333        33.333333   
4370186            50.000000         33.333333        33.333333   
4370187             0.000000          0.000000         0.000000   
4370188            40.000000         25.000000        25.000000   
4370189             0.000000          0.000000         0.000000   

         jaro_winkler_score  
0                  0.437037  
1                  0.688889  
2                  0.447619  
3                  0.455556  
4                  0.000000  
...                     ...  
4370185            0.583333  
4370186            0.583333  
4370187            0.000000  
4370188            0.500000  
4370189            0.000000  

[4370190 rows x 9 columns]

In [95]:
# fuzzy_words_df.describe()

original_index   match_index   ratio_score  partial_ratio_score  \
count    4.370190e+06  4.370190e+06  4.370190e+06         4.370190e+06   
mean     1.045000e+03  1.045000e+03  1.400943e+01         2.246970e+01   
std      6.036197e+02  6.036197e+02  1.502418e+01         2.266403e+01   
min      0.000000e+00  0.000000e+00  0.000000e+00         0.000000e+00   
25%      5.220000e+02  5.220000e+02  0.000000e+00         0.000000e+00   
50%      1.045000e+03  1.045000e+03  1.428571e+01         2.500000e+01   
75%      1.568000e+03  1.568000e+03  2.352941e+01         4.000000e+01   
max      2.090000e+03  2.090000e+03  9.629630e+01         1.000000e+02   

       token_sort_score  token_set_score  jaro_winkler_score  
count      4.370190e+06     4.370190e+06        4.370190e+06  
mean       1.400803e+01     1.400794e+01        2.240348e-01  
std        1.502212e+01     1.502198e+01        2.557097e-01  
min        0.000000e+00     0.000000e+00        0.000000e+00  
25%        0.000000e+00     0.000000e+00        0.000000e+00  
50%        1.428571e+01     1.428571e+01        0.000000e+00  
75%        2.352941e+01     2.352941e+01        4.666667e-01  
max        1.000000e+02     1.000000e+02        9.857143e-01

In [97]:
# fuzzy_words_df[(fuzzy_words_df['ratio_score']==0) & (fuzzy_words_df['partial_ratio_score']==0) & (fuzzy_words_df['token_sort_score']==0) & (fuzzy_words_df['token_set_score']==0) & (fuzzy_words_df['jaro_winkler_score']==0)]

original_index original_value  match_index match_value  ratio_score  \
4                     0          annex            5          po          0.0   
11                    0          annex           12       court          0.0   
14                    0          annex           15      lyford          0.0   
22                    0          annex           23       third          0.0   
23                    0          annex           24       floor          0.0   
...                 ...            ...          ...         ...          ...   
4370177            2090           2ntl         2077         rbc          0.0   
4370182            2090           2ntl         2082       hamas          0.0   
4370184            2090           2ntl         2084          zh          0.0   
4370187            2090           2ntl         2087         bav          0.0   
4370189            2090           2ntl         2089        cojp          0.0   

         partial_ratio_score  token_sort_score  token_set_score  \
4                        0.0               0.0              0.0   
11                       0.0               0.0              0.0   
14                       0.0               0.0              0.0   
22                       0.0               0.0              0.0   
23                       0.0               0.0              0.0   
...                      ...               ...              ...   
4370177                  0.0               0.0              0.0   
4370182                  0.0               0.0              0.0   
4370184                  0.0               0.0              0.0   
4370187                  0.0               0.0              0.0   
4370189                  0.0               0.0              0.0   

         jaro_winkler_score  
4                       0.0  
11                      0.0  
14                      0.0  
22                      0.0  
23                      0.0  
...                     ...  
4370177                 0.0  
4370182                 0.0  
4370184                 0.0  
4370187                 0.0  
4370189                 0.0  

[1911542 rows x 9 columns]

In [22]:
def calc_fuzz_df(df, column):
    row_list = []
    
    for o_i, o_v in enumerate(df[column].sort_index()):
        for m_i, m_v in enumerate(df[column].sort_index()):
            if o_i != m_i:
                dict1 = {
                    'original_index': o_i,
                    'original_value': o_v,
                    'match_index': m_i,
                    'match_value': m_v,
                    'ratio_score': rfuzz.ratio(o_v, m_v),
                    'partial_ratio_score': rfuzz.partial_ratio(o_v, m_v),
                    'token_sort_score': rfuzz.token_sort_ratio(o_v, m_v),
                    'token_set_score': rfuzz.token_set_ratio(o_v, m_v),
                    'jaro_winkler_score': jaro.jaro_winkler_metric(o_v, m_v)
                }
                if (dict1['ratio_score']>0) | (dict1['partial_ratio_score']>0) | (dict1['token_sort_score']>0) | (dict1['token_set_score']>0) | (dict1['jaro_winkler_score']>0):
                    row_list.append(dict1)
    score_df = pd.DataFrame(row_list)
        
    return score_df

In [23]:
fuzzy_words_df = calc_fuzz_df(freq_df, 'word')
fuzzy_words_df['jaro_winkler_score'] = fuzzy_words_df['jaro_winkler_score']*100
fuzzy_words_df

original_index original_value  match_index  match_value  ratio_score  \
0                     0          annex            1    frederick    14.285714   
1                     0          annex            2          and    50.000000   
2                     0          annex            3      shirley    16.666667   
3                     0          annex            4       street    18.181818   
4                     0          annex            6          box    25.000000   
...                 ...            ...          ...          ...          ...   
2275457            2039           2ntl         2030      tortola    36.363636   
2275458            2039           2ntl         2032  switzerland    26.666667   
2275459            2039           2ntl         2034     montagne    33.333333   
2275460            2039           2ntl         2035     sterline    33.333333   
2275461            2039           2ntl         2037         hast    25.000000   

         partial_ratio_score  token_sort_score  token_set_score  \
0                  25.000000         14.285714        14.285714   
1                  80.000000         50.000000        50.000000   
2                  28.571429         16.666667        16.666667   
3                  28.571429         18.181818        18.181818   
4                  50.000000         25.000000        25.000000   
...                      ...               ...              ...   
2275457            50.000000         36.363636        36.363636   
2275458            33.333333         26.666667        26.666667   
2275459            50.000000         33.333333        33.333333   
2275460            50.000000         33.333333        33.333333   
2275461            40.000000         25.000000        25.000000   

         jaro_winkler_score  
0                 43.703704  
1                 68.888889  
2                 44.761905  
3                 45.555556  
4                  0.000000  
...                     ...  
2275457           59.523810  
2275458           56.060606  
2275459           58.333333  
2275460           58.333333  
2275461           50.000000  

[2275462 rows x 9 columns]

In [55]:
index_col = 'original_index'
metric_cts = pd.DataFrame(fuzzy_words_df[index_col].unique(), columns=[index_col])

for metric in ['ratio_score', 'partial_ratio_score', 'token_sort_score', 'token_set_score', 'jaro_winkler_score']:
    met_df = fuzzy_words_df.loc[fuzzy_words_df[metric]>60, [index_col, metric]].groupby(index_col).count().reset_index()
    metric_cts = metric_cts.merge(met_df, on=index_col, how='outer')

In [56]:
metric_cts = fuzzy_words_df[[index_col, 'original_value']].drop_duplicates().merge(metric_cts, on=index_col, how='outer')
metric_cts.columns = ['original_index', 'original_value', 'ratio_match_ct', 'partial_ratio_match_ct', 'token_sort_match_ct', 'token_set_match_ct', 'jaro_winkler_match_ct']
metric_cts

original_index original_value  ratio_match_ct  partial_ratio_match_ct  \
0                  0          annex             5.0                      58   
1                  1      frederick             3.0                      63   
2                  2            and            14.0                     239   
3                  3        shirley            19.0                      60   
4                  4         street            18.0                      81   
...              ...            ...             ...                     ...   
2035            2035       sterline            21.0                     127   
2036            2036            bav             7.0                      98   
2037            2037           hast             6.0                     110   
2038            2038            coj             1.0                     100   
2039            2039           2ntl             NaN                      52   

      token_sort_match_ct  token_set_match_ct  jaro_winkler_match_ct  
0                     5.0                 5.0                     55  
1                     3.0                 3.0                     67  
2                    14.0                14.0                    133  
3                    19.0                19.0                    122  
4                    18.0                18.0                    179  
...                   ...                 ...                    ...  
2035                 21.0                21.0                    238  
2036                  7.0                 7.0                     91  
2037                  6.0                 6.0                    114  
2038                  1.0                 1.0                     84  
2039                  NaN                 NaN                     37  

[2040 rows x 7 columns]

In [223]:
# metric_cts[metric_cts['original_value'].str.contains('^n\d+|cb\d+|no\d+|\d+$')]

original_index original_value  ratio_match_ct  partial_ratio_match_ct  \
7                  7          n4805            97.0                   238.0   
14                14          n8188            66.0                   225.0   
18                18          n7785            81.0                   198.0   
19                19          n3708            89.0                   271.0   
25                25          n3024           100.0                   267.0   
26                26            303            31.0                   122.0   
27                27           n492            75.0                   237.0   
31                31        ss19084            25.0                   124.0   
37                37        cb12399            52.0                   143.0   
43                43          n4875           115.0                   268.0   
49                49           3933            15.0                   105.0   
51                51        ap59213             9.0                   121.0   
53                53            308            26.0                   119.0   
55                55          n7768            56.0                   190.0   
60                60          n4755            74.0                   227.0   
61                61              1            21.0                   391.0   
63                63          n7117            83.0                   251.0   
65                65              4            14.0                   274.0   
67                67          n3937            80.0                   241.0   
70                70         ss5383            17.0                   113.0   
75                75         n10429           148.0                   276.0   
76                76          n3242           100.0                   273.0   
79                79          n9100            83.0                   235.0   
82                82          13050            44.0                   187.0   
85                85          n3231            77.0                   272.0   
86                86          n3023            85.0                   208.0   
87                87            529            21.0                    97.0   
88                88        ss19195            20.0                   118.0   
90                90         gt2503             4.0                    71.0   
91                91          n4837           106.0                   305.0   
92                92            680            14.0                    79.0   
93                93          n8199            77.0                   245.0   
94                94           n341            75.0                   324.0   
95                95          n4465            66.0                   181.0   
96                96          n9306            80.0                   228.0   
97                97          n1113            55.0                   261.0   
98                98          n1000            49.0                   195.0   
102              102          n3930            80.0                   213.0   
104              104          63131            22.0                   146.0   
107              107           n529            33.0                   197.0   
108              108            102            54.0                   222.0   
111              111        cb13937            58.0                   154.0   
115              115          n1965            87.0                   225.0   
116              116          n7525            61.0                   194.0   
117              117          n4906            94.0                   207.0   
118              118          n7549            89.0                   241.0   
121              121        cb11343            57.0                   175.0   
122              122          n1682           102.0                   269.0   
123              123         ss5539            11.0                   124.0   
124              124          n7940            88.0     

In [224]:
# metric_cts[metric_cts['original_value'].str.contains('^n\d+|cb\d+|no\d+|\d+$')].shape

(785, 7)

In [228]:
# metric_cts[metric_cts['original_value'].str.contains('^n\d+|cb\d+|no\d+|^\d+$')].shape

(629, 7)

In [225]:
# metric_cts[metric_cts['original_value'].str.contains('\d+$')].shape

(784, 7)

In [227]:
# metric_cts[metric_cts['original_value'].str.contains('^\d+$')].shape

(247, 7)

In [59]:
metric_cts = metric_cts[~metric_cts['original_value'].str.contains('^n\d+|^\w\w\d+|^\d+$')]
metric_cts

original_index original_value  ratio_match_ct  partial_ratio_match_ct  \
0                  0          annex             5.0                      58   
1                  1      frederick             3.0                      63   
2                  2            and            14.0                     239   
3                  3        shirley            19.0                      60   
4                  4         street            18.0                      81   
...              ...            ...             ...                     ...   
2035            2035       sterline            21.0                     127   
2036            2036            bav             7.0                      98   
2037            2037           hast             6.0                     110   
2038            2038            coj             1.0                     100   
2039            2039           2ntl             NaN                      52   

      token_sort_match_ct  token_set_match_ct  jaro_winkler_match_ct  
0                     5.0                 5.0                     55  
1                     3.0                 3.0                     67  
2                    14.0                14.0                    133  
3                    19.0                19.0                    122  
4                    18.0                18.0                    179  
...                   ...                 ...                    ...  
2035                 21.0                21.0                    238  
2036                  7.0                 7.0                     91  
2037                  6.0                 6.0                    114  
2038                  1.0                 1.0                     84  
2039                  NaN                 NaN                     37  

[1252 rows x 7 columns]

In [60]:
metric_cts.sort_values('ratio_match_ct', ascending=False).head(1000)

original_index   original_value  ratio_match_ct  partial_ratio_match_ct  \
1137            1137              ste            24.0                     243   
516              516           bahams            24.0                      71   
1136            1136            slite            23.0                      94   
1580            1580           steret            23.0                     113   
662              662           center            23.0                     100   
659              659           bhamas            23.0                      76   
1226            1226         charlote            23.0                      92   
904              904           bahmas            22.0                      68   
1969            1969             stre            22.0                     205   
464              464           suites            22.0                      80   
947              947           strees            22.0                      87   
9                  9          bahamas            22.0                      76   
1630            1630           strets            21.0                      87   
2031            2031            hamas            21.0                      75   
761              761             land            21.0                     143   
986              986            paton            21.0                      84   
188              188           bahama            21.0                      78   
914              914           ahamas            21.0                      72   
2035            2035         sterline            21.0                     127   
948              948         carlotte            20.0                      79   
1018            1018          bahaams            20.0                      65   
282              282             port            20.0                     108   
1124            1124          bahanas            20.0                      87   
1437            1437            elite            20.0                      89   
112              112             lane            20.0                     179   
607              607            stree            19.0                      89   
3                  3          shirley            19.0                      60   
1603            1603         bahamasa            19.0                      89   
1228            1228          brahmas            19.0                      70   
10                10            suite            19.0                      83   
907              907         bahamasc            18.0                      69   
1125            1125        bahamaspo            18.0                      71   
1912            1912         abahamas            18.0                      80   
1932            1932            baham            18.0                      65   
949              949         chalotte            18.0                      72   
930              930         bahamas1            18.0                      79   
1598            1598            maria            18.0                      80   
1201            1201         bahamas6            18.0                      70   
456              456             fort            18.0                      86   
4                  4           street            18.0                      81   
1101            1101         bahamaas            18.0                      78   
911              911         shorline            17.0                     103   
1184            1184    nassaubahamas            17.0                      90   
1087            1087          shorley            17.0                      55   
845              845          cavenue            17.0                      72   
22                22           centre            17.0                      93   
280              280           marina            17.0                      95   
1309            1309         balvenie            16.0                      87   
1093            1093            lance

In [28]:
metric_cts.sort_values('ratio_match_ct', ascending=False).tail(400)

original_index          original_value  ratio_match_ct  \
971              971                   bitco            13.0   
765              765                 ee15269            13.0   
581              581                 ab20763            13.0   
73                73             marlborough            13.0   
970              970                     bit            13.0   
705              705                halsbury            13.0   
74                74                   queen            13.0   
1181            1181                  2union            13.0   
1259            1259                   wueen            13.0   
1460            1460                 olimpia            13.0   
361              361                 ee17971            13.0   
175              175                  equity            12.0   
883              883                     kln            12.0   
1970            1970                      et            12.0   
1210            1210              elizzabeth            12.0   
744              744                 malcolm            12.0   
445              445                     ltd            12.0   
341              341                   fifth            12.0   
1316            1316               bostonian            12.0   
1941            1941                     fes            12.0   
400              400                 heights            12.0   
241              241                  f60361            12.0   
297              297                  f60452            12.0   
1948            1948                    with            12.0   
1257            1257                    qeen            12.0   
1287            1287                   sugar            12.0   
1278            1278               adminotis            12.0   
1946            1946                   buldg            12.0   
1616            1616                    cibc            12.0   
932              932                 address            12.0   
1266            1266               worldwide            12.0   
1680            1680                  harrow            12.0   
1943            1943                   limit            12.0   
643              643                  ss5062            12.0   
480              480                 cooplaw            12.0   
1208            1208                elizabth            12.0   
1374            1374                   ceiba            12.0   
991              991                  taylor            12.0   
559              559                 sp64284            12.0   
1497            1497            developments            12.0   
2016            2016                    love            12.0   
1620            1620                 academy            12.0   
1435            1435                limitada            12.0   
966              966                     n°1            12.0   
1838            1838                   kelly            12.0   
826              826                 ee15790            12.0   
752              752               evergreen            12.0   
1821            1821                 william            12.0   
946              946                  geneva            12.0   
1372            1372                 anónima            12.0   
1047            1047                 gregory            12.0   
1817            1817                   pilot            12.0   
16                16                  lyford            12.0   
522              522                   toote            12.0   
1355            1355                 ecuador            12.0   
1457            1457          administrators            12.0   
718              718                       b            12.0   
558              558                beaumont            12.0   
1660            1660               muirfield            11.0   
1684            1684                 meadows            11.0   
901              901                 begawan            11.0   
984              984           neville

In [51]:
df[df['address_wordlist'].apply(lambda x: 'st' in x)]

node_id  \
450    24000451   
960    14078480   
961    14078481   
1935   33000132   
2161  120015591   

                                                                            address  \
450     ST. MALCOLM BUILDING, VICTORIA & BAY STS, P.O. BOX SS-6738, NASSAU, BAHAMAS   
960                     St. Andrew's Court; Frederick Street Steps; Nassau; Bahamas   
961   St. Andrew's Court; Frederick Street Steps; P. O. Box N-4805; Nassau; Bahamas   
1935            ST ANDREW'S COURT FREDERICK ST STEPS PO BOX N-4805, NASSAU, BAHAMAS   
2161                     ST. ANDREW'S COURT FREDERICK STREET, STEPS, NASSAU BAHAMAS   

                                                            name countries  \
450                                                          NaN   Bahamas   
960                                                          NaN   Bahamas   
961                                                          NaN   Bahamas   
1935                                                         NaN   Bahamas   
2161  ST. ANDREW'S COURT FREDERICK STREET, STEPS, NASSAU BAHAMAS   Bahamas   

     country_codes                                       sourceID  \
450            BHS                                  Bahamas Leaks   
960            BHS                                  Panama Papers   
961            BHS                                  Panama Papers   
1935           BHS   Paradise Papers - Bahamas corporate registry   
2161           BHS  Paradise Papers - Barbados corporate registry   

                                                   valid_until note  \
450      The Bahamas Leaks data is current through early 2016.  NaN   
960            The Panama Papers  data is current through 2015  NaN   
961            The Panama Papers  data is current through 2015  NaN   
1935   Bahamas corporate registry data is current through 2016  NaN   
2161  Barbados corporate registry data is current through 2016  NaN   

                                                               working_address  \
450   st malcolm building victoria and bay street po box ss6738 nassau bahamas   
960                     st andrews court frederick street steps nassau bahamas   
961       st andrews court frederick street steps p o box n4805 nassau bahamas   
1935       st andrews court frederick street steps po box n4805 nassau bahamas   
2161                    st andrews court frederick street steps nassau bahamas   

                                                                           address_wordlist  
450   [st, malcolm, building, victoria, and, bay, street, po, box, ss6738, nassau, bahamas]  
960                         [st, andrews, court, frederick, street, steps, nassau, bahamas]  
961       [st, andrews, court, frederick, street, steps, p, o, box, n4805, nassau, bahamas]  
1935        [st, andrews, court, frederick, street, steps, po, box, n4805, nassau, bahamas]  
2161                        [st, andrews, court, frederick, street, steps, nassau, bahamas]

In [54]:
df[df['address_wordlist'].apply(lambda x: 'corporate' in x)]

node_id  \
9      24000010   
63     24000064   
100    24000101   
116    24000117   
227    24000228   
248    24000249   
268    24000269   
301    24000302   
476    24000477   
483    24000484   
548    14012315   
661    14032773   
756    14043534   
758    14043536   
759    14043537   
760    14043538   
761    14043539   
762    14043540   
763    14043541   
764    14043542   
765    14043543   
766    14043544   
767    14043545   
768    14043546   
769    14043547   
770    14043548   
771    14043549   
788    14047561   
1197   14090235   
1315   14104053   
1414     268653   
1459     263636   
1511   81017514   
1524   81019465   
1532   81021515   
1564   81024997   
1591   81031328   
1610   81036217   
1667   81056984   
1707   81071850   
1713   81074260   
1789   81012499   
1800   81013670   
1975   33000175   
2052   33000259   
2066   33000273   
2068   33000275   
2093   33000301   
2120   33000330   
2127  120005466   
2128  120002895   
2133  120004101   
2143  120010247   
2157  120009681   
2162  120017256   
2211  240450230   
2216  240491955   

                                                                                                                                           address  \
9                                                                       GROUND FLOOR, GOODMAN'S BAY CORPORATE CE, P.O. BOX N 3933, NASSAU, BAHAMAS   
63                                                                       GOODMAN'S BAY CORPORATE CENTRE WEST BAY, P.O. BOX N-3015, NASSAU, BAHAMAS   
100                                                             GOODMAN'S BAY CORPORATE CENTRE, P.O. BOX CB-10976 WEST BAY STREET, NASSAU, BAHAMAS   
116                                                                    GOODMANS BAY CORPORATE CENTRE SUITE 261, P.O. BOX CB-12762, NASSAU, BAHAMAS   
227                                                                          C/O H & J CORPORATE SERVICES LTD., P.O. BOX CB-13278, NASSAU, BAHAMAS   
248                                                       GOODMAN'S BAY CORPORATE CENTRE, P.O. BOX SS-5498 SUITE# 261 WEST BAY ST, NASSAU, BAHAMAS   
268                                                                              GOODMANS BAY CORPORATE CENTRE, P.O. BOX CB-12407, NASSAU, BAHAMAS   
301                                                      P.O. BOX SS-5800, PRIDEROCK CORPORATE CENTRE, SUITE 200, BAY & EAST STS., NASSAU, BAHAMAS   
476                                                                   GOODMAN'S CORPORATE CENTER, WEST BAY ST., P.O. BOX SP-61567, NASSAU, BAHAMAS   
483                                                                    SUITE 6A JASMINE CORPORATE CENTRE, P.O. BOX F-402823 FREEPORT, GRAND BAHAMA   
548                                                      2ND  FLOOR; GOODMAN'S BAY CORPORATE CENTRE SUITE #261; P.O. BOX CB-12762; NASSAU; BAHAMAS   
661                                c/o GTC Corporate Services Limited; Sassoon House; Shirley St. & Victoria Ave.; PO Box SS-5383; Nassau; Bahamas   
756                                                                   GOODMAN0S BAY CORPORATE CENTRE, WEST BAY STREET PO BOX N-4938 NASSAU BAHAMAS   
758                                                                Goodman's Bay Corporate Center; PO Box N-3225; West Bay Street; Nassau; Bahamas   
759                                                         GOODMAN S BAY CORPORATE CENTER - WEST BAY STREET AND SEA VIEW DRIVE - NASSAU - BAHAMAS   
760                                                                                    GOODMAN S  BAY CORPORATE CENTER WEST BAY STREET NASSAU  BAH   
761                                       Goodman's Bay Corporate Centre; 3rd Floor; West Bay Street and Sea View Drive; CB-10976; Nassau; Bahamas   
762                                                                      GOODMAN'S BAY CORPORATE CENTRE THIRD FLOOR WEST BAY STREET NASSAU BAHAMAS   
763                                                    

In [61]:
df[df['address_wordlist'].apply(lambda x: 'bahams' in x)]

node_id  \
274   24000275   
559   14018044   
631   14030207   
826   14050608   
867   14064246   
889   14064268   
1910  33000104   

                                                                                                                               address  \
274                                                                                                    P.O. BOX N 8680, NASSAU, BAHAMS   
559                                                    4TH FLOOR THE BAHAMAS FINANCIAL CENTRE SHIRLEY & CHARLOTTE STREET NASSAU BAHAMS   
631                                                    BAHAMS FINANCILA CENTRE PO BOX N-3023 SHIRLEY & CHARLOTTE STREETSNASSAU BAHAMAS   
826   MOSSACK FONSECA & CO (BAHAMS) LIMITED SAFFREY SQUARE; SUITE 205; BANK LANE; P.O.BOX N-8188; NASSAU; COMMONWEALTH OF THE BAHAMAS.   
867                                                                 P.O.BOX N-3944; PROVIDENCE HOUSE; EAST HILL STREET; NASSAU; BAHAMS   
889                                                                                                        P O BOX N8188 NASSAU BAHAMS   
1910                                                                                                                    NASSAU, BAHAMS   

     name countries country_codes  \
274   NaN   Bahamas           BHS   
559   NaN   Bahamas           BHS   
631   NaN   Bahamas           BHS   
826   NaN   Bahamas           BHS   
867   NaN   Bahamas           BHS   
889   NaN   Bahamas           BHS   
1910  NaN   Bahamas           BHS   

                                          sourceID  \
274                                  Bahamas Leaks   
559                                  Panama Papers   
631                                  Panama Papers   
826                                  Panama Papers   
867                                  Panama Papers   
889                                  Panama Papers   
1910  Paradise Papers - Bahamas corporate registry   

                                                  valid_until note  \
274     The Bahamas Leaks data is current through early 2016.  NaN   
559           The Panama Papers  data is current through 2015  NaN   
631           The Panama Papers  data is current through 2015  NaN   
826           The Panama Papers  data is current through 2015  NaN   
867           The Panama Papers  data is current through 2015  NaN   
889           The Panama Papers  data is current through 2015  NaN   
1910  Bahamas corporate registry data is current through 2016  NaN   

                                                                                                                working_address  \
274                                                                                                 po box n 8680 nassau bahams   
559                                        fourth floor the bahamas financial centre shirley and charlotte street nassau bahams   
631                                            bahams financila centre po box n3023 shirley and charlotte street nassau bahamas   
826   mossack fonseca and co bahams limited saffrey square suite 205 bank lane po box n8188 nassau commonwealth of the bahamas    
867                                                                po box n3944 providence house east hill street nassau bahams   
889                                                                                                  po box n8188 nassau bahams   
1910                                                                                                              nassau bahams   

                                                                                                                                   address_wordlist  
274                                                                                                              [po, box, n, 8680, nassau, bahams]  
559                                               [fourth, floor, the, bahamas, financial, centre, shirley, and, char

In [75]:
df[df['address_wordlist'].apply(lambda x: 'bhs' in x)]

node_id  \
594   14030170   
1102  14080652   

                                                                          address  \
594   Bahamas Financial Center; Charlotte & Shirley Streets; Nassau; Bahamas; BHS   
1102       The Bahamas Financial Centre; Shirley & Charlotte Streets; Nassau; BHS   

     name countries country_codes       sourceID  \
594   NaN   Bahamas           BHS  Panama Papers   
1102  NaN   Bahamas           BHS  Panama Papers   

                                          valid_until note  \
594   The Panama Papers  data is current through 2015  NaN   
1102  The Panama Papers  data is current through 2015  NaN   

                                                               working_address  \
594   bahamas financial center charlotte and shirley street nassau bahamas bhs   
1102      the bahamas financial centre shirley and charlotte street nassau bhs   

                                                                         address_wordlist  
594   [bahamas, financial, center, charlotte, and, shirley, street, nassau, bahamas, bhs]  
1102      [the, bahamas, financial, centre, shirley, and, charlotte, street, nassau, bhs]

In [82]:
fuzzy_words_df[(fuzzy_words_df['original_value']=='street') & (fuzzy_words_df['ratio_score']>50)].sort_values('ratio_score', ascending=False)

original_index original_value  match_index match_value  ratio_score  \
5572               4         street         1199     streeet    92.307692   
5174               4         street          322       stret    90.909091   
5278               4         street          607       stree    90.909091   
5997               4         street         1791    streeets    85.714286   
5906               4         street         1630      strets    83.333333   
5559               4         street         1179      streer    83.333333   
5874               4         street         1580      steret    83.333333   
5431               4         street          947      strees    83.333333   
6088               4         street         1969        stre    80.000000   
5975               4         street         1744    treetops    71.428571   
5535               4         street         1137         ste    66.666667   
5134               4         street          197      regent    66.666667   
6025               4         street         1837      secret    66.666667   
5956               4         street         1714  settlement    62.500000   
5197               4         street          383  retirement    62.500000   
5623               4         street         1271     trustee    61.538462   
6021               4         street         1831     trident    61.538462   
5269               4         street          584     seventh    61.538462   
5541               4         street         1153        sede    60.000000   
6112               4         street         2010        reef    60.000000   
5971               4         street         1739    treasure    57.142857   
6001               4         street         1797    stephane    57.142857   
6127               4         street         2035    sterline    57.142857   
5416               4         street          927    trustees    57.142857   
5822               4         street         1512    software    57.142857   
5128               4         street          186    freeport    57.142857   
5546               4         street         1158       steps    54.545455   
5209               4         street          407       trade    54.545455   
5322               4         street          738       trehl    54.545455   
5998               4         street         1793       moree    54.545455   
5534               4         street         1136       slite    54.545455   
5828               4         street         1519       sport    54.545455   
5584               4         street         1212       sutie    54.545455   
5124               4         street          176       trust    54.545455   
5049               4         street           10       suite    54.545455   
5832               4         street         1523   starshine    53.333333   
5927               4         street         1668   summerset    53.333333   
5911               4         street         1638   shoreline    53.333333   
5280               4         street          615   seventeen    53.333333   
6032               4         street         1851   seabreeze    53.333333   
6061               4         street         1898   sommerset    53.333333   
5888               4         street         1602   westridge    53.333333   
5127               4         street          181   templeton    53.333333   

      partial_ratio_score  token_sort_score  token_set_score  \
5572            90.909091         92.307692        92.307692   
5174            88.888889         90.909091        90.909091   
5278           100.000000         90.909091        90.909091   
5997            90.909091         85.714286        85.714286   
5906            90.909091         83.333333        83.333333   
5559            90.909091         83.333333        83.333333   
5874            83.333333         83.333333        83.333333   
5431            90.909091         83.333333        83.333333   
6088           100.000000         80.0000

In [83]:
fuzzy_words_df[(fuzzy_words_df['original_value']=='street') & (fuzzy_words_df['partial_ratio_score']>50)].sort_values('partial_ratio_score', ascending=False)

original_index original_value  match_index    match_value  ratio_score  \
5310               4         street          713             ee    50.000000   
5979               4         street         1752              t    28.571429   
5215               4         street          433              r    28.571429   
5278               4         street          607          stree    90.909091   
5104               4         street          123              s    28.571429   
...              ...            ...          ...            ...          ...   
5764               4         street         1443        systems    46.153846   
5763               4         street         1442        netware    46.153846   
5752               4         street         1430   microsystems    33.333333   
5713               4         street         1381       ventures    42.857143   
5684               4         street         1344  distribuidora    31.578947   

      partial_ratio_score  token_sort_score  token_set_score  \
5310           100.000000         50.000000        50.000000   
5979           100.000000         28.571429        28.571429   
5215           100.000000         28.571429        28.571429   
5278           100.000000         90.909091        90.909091   
5104           100.000000         28.571429        28.571429   
...                   ...               ...              ...   
5764            54.545455         46.153846        46.153846   
5763            54.545455         46.153846        46.153846   
5752            54.545455         33.333333        33.333333   
5713            54.545455         42.857143        42.857143   
5684            54.545455         31.578947        31.578947   

      jaro_winkler_score  
5310           55.555556  
5979           72.222222  
5215           72.222222  
5278           96.666667  
5104           75.000000  
...                  ...  
5764           64.285714  
5763           53.174603  
5752           50.000000  
5713           63.888889  
5684           57.692308  

[191 rows x 9 columns]

In [84]:
fuzzy_words_df[(fuzzy_words_df['original_value']=='street') & (fuzzy_words_df['token_sort_score']>50)].sort_values('token_sort_score', ascending=False)

original_index original_value  match_index match_value  ratio_score  \
5572               4         street         1199     streeet    92.307692   
5174               4         street          322       stret    90.909091   
5278               4         street          607       stree    90.909091   
5997               4         street         1791    streeets    85.714286   
5906               4         street         1630      strets    83.333333   
5559               4         street         1179      streer    83.333333   
5874               4         street         1580      steret    83.333333   
5431               4         street          947      strees    83.333333   
6088               4         street         1969        stre    80.000000   
5975               4         street         1744    treetops    71.428571   
5535               4         street         1137         ste    66.666667   
5134               4         street          197      regent    66.666667   
6025               4         street         1837      secret    66.666667   
5956               4         street         1714  settlement    62.500000   
5197               4         street          383  retirement    62.500000   
5623               4         street         1271     trustee    61.538462   
6021               4         street         1831     trident    61.538462   
5269               4         street          584     seventh    61.538462   
5541               4         street         1153        sede    60.000000   
6112               4         street         2010        reef    60.000000   
5971               4         street         1739    treasure    57.142857   
6001               4         street         1797    stephane    57.142857   
6127               4         street         2035    sterline    57.142857   
5416               4         street          927    trustees    57.142857   
5822               4         street         1512    software    57.142857   
5128               4         street          186    freeport    57.142857   
5546               4         street         1158       steps    54.545455   
5209               4         street          407       trade    54.545455   
5322               4         street          738       trehl    54.545455   
5998               4         street         1793       moree    54.545455   
5534               4         street         1136       slite    54.545455   
5828               4         street         1519       sport    54.545455   
5584               4         street         1212       sutie    54.545455   
5124               4         street          176       trust    54.545455   
5049               4         street           10       suite    54.545455   
5832               4         street         1523   starshine    53.333333   
5927               4         street         1668   summerset    53.333333   
5911               4         street         1638   shoreline    53.333333   
5280               4         street          615   seventeen    53.333333   
6032               4         street         1851   seabreeze    53.333333   
6061               4         street         1898   sommerset    53.333333   
5888               4         street         1602   westridge    53.333333   
5127               4         street          181   templeton    53.333333   

      partial_ratio_score  token_sort_score  token_set_score  \
5572            90.909091         92.307692        92.307692   
5174            88.888889         90.909091        90.909091   
5278           100.000000         90.909091        90.909091   
5997            90.909091         85.714286        85.714286   
5906            90.909091         83.333333        83.333333   
5559            90.909091         83.333333        83.333333   
5874            83.333333         83.333333        83.333333   
5431            90.909091         83.333333        83.333333   
6088           100.000000         80.0000

In [85]:
fuzzy_words_df[(fuzzy_words_df['original_value']=='street') & (fuzzy_words_df['token_set_score']>50)].sort_values('token_set_score', ascending=False)

original_index original_value  match_index match_value  ratio_score  \
5572               4         street         1199     streeet    92.307692   
5174               4         street          322       stret    90.909091   
5278               4         street          607       stree    90.909091   
5997               4         street         1791    streeets    85.714286   
5906               4         street         1630      strets    83.333333   
5559               4         street         1179      streer    83.333333   
5874               4         street         1580      steret    83.333333   
5431               4         street          947      strees    83.333333   
6088               4         street         1969        stre    80.000000   
5975               4         street         1744    treetops    71.428571   
5535               4         street         1137         ste    66.666667   
5134               4         street          197      regent    66.666667   
6025               4         street         1837      secret    66.666667   
5956               4         street         1714  settlement    62.500000   
5197               4         street          383  retirement    62.500000   
5623               4         street         1271     trustee    61.538462   
6021               4         street         1831     trident    61.538462   
5269               4         street          584     seventh    61.538462   
5541               4         street         1153        sede    60.000000   
6112               4         street         2010        reef    60.000000   
5971               4         street         1739    treasure    57.142857   
6001               4         street         1797    stephane    57.142857   
6127               4         street         2035    sterline    57.142857   
5416               4         street          927    trustees    57.142857   
5822               4         street         1512    software    57.142857   
5128               4         street          186    freeport    57.142857   
5546               4         street         1158       steps    54.545455   
5209               4         street          407       trade    54.545455   
5322               4         street          738       trehl    54.545455   
5998               4         street         1793       moree    54.545455   
5534               4         street         1136       slite    54.545455   
5828               4         street         1519       sport    54.545455   
5584               4         street         1212       sutie    54.545455   
5124               4         street          176       trust    54.545455   
5049               4         street           10       suite    54.545455   
5832               4         street         1523   starshine    53.333333   
5927               4         street         1668   summerset    53.333333   
5911               4         street         1638   shoreline    53.333333   
5280               4         street          615   seventeen    53.333333   
6032               4         street         1851   seabreeze    53.333333   
6061               4         street         1898   sommerset    53.333333   
5888               4         street         1602   westridge    53.333333   
5127               4         street          181   templeton    53.333333   

      partial_ratio_score  token_sort_score  token_set_score  \
5572            90.909091         92.307692        92.307692   
5174            88.888889         90.909091        90.909091   
5278           100.000000         90.909091        90.909091   
5997            90.909091         85.714286        85.714286   
5906            90.909091         83.333333        83.333333   
5559            90.909091         83.333333        83.333333   
5874            83.333333         83.333333        83.333333   
5431            90.909091         83.333333        83.333333   
6088           100.000000         80.0000

In [86]:
fuzzy_words_df[(fuzzy_words_df['original_value']=='street') & (fuzzy_words_df['jaro_winkler_score']>50)].sort_values('jaro_winkler_score', ascending=False)

original_index original_value  match_index  match_value  ratio_score  \
5572               4         street         1199      streeet    92.307692   
5174               4         street          322        stret    90.909091   
5278               4         street          607        stree    90.909091   
5874               4         street         1580       steret    83.333333   
5997               4         street         1791     streeets    85.714286   
...              ...            ...          ...          ...          ...   
5792               4         street         1479  consultores    47.058824   
5651               4         street         1305  consultants    35.294118   
5556               4         street         1171  redomiciled    35.294118   
5695               4         street         1357  association    23.529412   
5342               4         street          784  sandringham    23.529412   

      partial_ratio_score  token_sort_score  token_set_score  \
5572            90.909091         92.307692        92.307692   
5174            88.888889         90.909091        90.909091   
5278           100.000000         90.909091        90.909091   
5874            83.333333         83.333333        83.333333   
5997            90.909091         85.714286        85.714286   
...                   ...               ...              ...   
5792            54.545455         47.058824        47.058824   
5651            36.363636         35.294118        35.294118   
5556            50.000000         35.294118        35.294118   
5695            33.333333         23.529412        23.529412   
5342            36.363636         23.529412        23.529412   

      jaro_winkler_score  
5572           97.142857  
5174           96.666667  
5278           96.666667  
5874           95.555556  
5997           95.000000  
...                  ...  
5792           50.505051  
5651           50.505051  
5556           50.505051  
5695           50.505051  
5342           50.505051  

[484 rows x 9 columns]

In [92]:
word_list = fuzzy_words_df[(fuzzy_words_df['original_value']=='street') & (fuzzy_words_df['jaro_winkler_score']>75)].sort_values('jaro_winkler_score', ascending=False)['match_value']
for word in word_list:
    print(word)
    display(df[df['address_wordlist'].apply(lambda x: word in x)])
    print('\n')

bahamasc


node_id                                                 address name  \
570  14018521  51 Frederick Street; P.O. Box N-1136; Nassau; BahamasC  NaN   

    countries country_codes       sourceID  \
570   Bahamas           BHS  Panama Papers   

                                         valid_until note  \
570  The Panama Papers  data is current through 2015  NaN   

                                      working_address  \
570  51 frederick street po box n1136 nassau bahamasc   

                                              address_wordlist  
570  [51, frederick, street, po, box, n1136, nassau, bahamasc]



bahamasa


node_id  \
1442   240054   

                                                                              address  \
1442  Winterbotham Place Marlborough & Queen Streets PO Box CB 11343 Nassau, Bahamasa   

     name countries country_codes        sourceID  \
1442  NaN   Bahamas           BHS  Offshore Leaks   

                                          valid_until note  \
1442  The Offshore Leaks data is current through 2010  NaN   

                                                                      working_address  \
1442  winterbotham place marlborough and queen street po box cb 11343 nassau bahamasa   

                                                                                  address_wordlist  
1442  [winterbotham, place, marlborough, and, queen, street, po, box, cb, 11343, nassau, bahamasa]



bahamas6


...

node_id  \
549  14012324   

                                                                            address  \
549  2nd Floor; International Bazaar; Bay Street; P.O. Box N- 1612; Nassau; Bahamas   

    name countries country_codes       sourceID  \
549  NaN   Bahamas           BHS  Panama Papers   

                                         valid_until note  \
549  The Panama Papers  data is current through 2015  NaN   

                                                               working_address  \
549  second floor international bazaar bay street po box n 1612 nassau bahamas   

                                                                           address_wordlist  
549  [second, floor, international, bazaar, bay, street, po, box, n, 1612, nassau, bahamas]



bosham


node_id  \
199    24000200   
1887   33000077   
2180  240003759   

                                                                     address  \
199   #6 BOSHAM CLOSE, CAMPERDOWN HEIGHTS P.O. BOX SP 63801, NASSAU, BAHAMAS   
1887    #6 BOSHAM CLOSE, CAMPERDOWN HEIGHTS PO BOX SP 63801, NASSAU, BAHAMAS   
2180           NO. 6 BOSHAM CLOSE, CAMPERDOWN HEIGHTS NEW PROVIDENCE BAHAMAS   

     name countries country_codes  \
199   NaN   Bahamas           BHS   
1887  NaN   Bahamas           BHS   
2180  NaN   Bahamas           BHS   

                                                       sourceID  \
199                                               Bahamas Leaks   
1887               Paradise Papers - Bahamas corporate registry   
2180  Pandora Papers - Alemán, Cordero, Galindo & Lee (Alcogal)   

                                                  valid_until note  \
199     The Bahamas Leaks data is current through early 2016.  NaN   
1887  Bahamas corporate registry data is current through 2016  NaN   
2180                    Provider data is current through 2018  NaN   

                                                       working_address  \
199   6 bosham close camperdown heights po box sp 63801 nassau bahamas   
1887  6 bosham close camperdown heights po box sp 63801 nassau bahamas   
2180       no 6 bosham close camperdown heights new providence bahamas   

                                                                  address_wordlist  
199   [6, bosham, close, camperdown, heights, po, box, sp, 63801, nassau, bahamas]  
1887  [6, bosham, close, camperdown, heights, po, box, sp, 63801, nassau, bahamas]  
2180         [no, 6, bosham, close, camperdown, heights, new, providence, bahamas]



hamas


node_id  \
2243  240492153   

                                                                             address  \
2243  MONTAGUE STERLING CENTRE. EAST BAY STREET, NASSAU, HAMAS, SWITZERLAND, BAHAMAS   

     name countries country_codes                        sourceID  \
2243  NaN   Bahamas           BHS  Pandora Papers - Trident Trust   

                                valid_until note  \
2243  Provider data is current through 2016  NaN   

                                                                working_address  \
2243  montague sterling centre east bay street nassau hamas switzerland bahamas   

                                                                          address_wordlist  
2243  [montague, sterling, centre, east, bay, street, nassau, hamas, switzerland, bahamas]

### Bahamas

In [87]:
fuzzy_words_df[(fuzzy_words_df['original_value']=='bahamas') & (fuzzy_words_df['ratio_score']>60)].sort_values('ratio_score', ascending=False)

original_index original_value  match_index    match_value  ratio_score  \
10611               9        bahamas         1201       bahamas6    93.333333   
11095               9        bahamas         1912       abahamas    93.333333   
10428               9        bahamas          907       bahamasc    93.333333   
10443               9        bahamas          930       bahamas1    93.333333   
10917               9        bahamas         1603       bahamasa    93.333333   
10551               9        bahamas         1101       bahamaas    93.333333   
10114               9        bahamas          188         bahama    92.307692   
10240               9        bahamas          516         bahams    92.307692   
10307               9        bahamas          659         bhamas    92.307692   
10427               9        bahamas          904         bahmas    92.307692   
10431               9        bahamas          914         ahamas    92.307692   
10558               9        bahamas         1125      bahamaspo    87.500000   
10631               9        bahamas         1228        brahmas    85.714286   
10557               9        bahamas         1124        bahanas    85.714286   
10498               9        bahamas         1018        bahaams    85.714286   
11100               9        bahamas         1932          baham    83.333333   
11153               9        bahamas         2031          hamas    83.333333   
10596               9        bahamas         1184  nassaubahamas    70.000000   
10197               9        bahamas          397         bosham    61.538462   
10701               9        bahamas         1321         champs    61.538462   
10981               9        bahamas         1709         thomas    61.538462   
10416               9        bahamas          885         bazaar    61.538462   

       partial_ratio_score  token_sort_score  token_set_score  \
10611           100.000000         93.333333        93.333333   
11095           100.000000         93.333333        93.333333   
10428           100.000000         93.333333        93.333333   
10443           100.000000         93.333333        93.333333   
10917           100.000000         93.333333        93.333333   
10551            92.307692         93.333333        93.333333   
10114           100.000000         92.307692        92.307692   
10240            90.909091         92.307692        92.307692   
10307            90.909091         92.307692        92.307692   
10427            83.333333         92.307692        92.307692   
10431           100.000000         92.307692        92.307692   
10558           100.000000         87.500000        87.500000   
10631            85.714286         85.714286        85.714286   
10557            85.714286         85.714286        85.714286   
10498            85.714286         85.714286        85.714286   
11100           100.000000         83.333333        83.333333   
11153           100.000000         83.333333        83.333333   
10596           100.000000         70.000000        70.000000   
10197            72.727273         61.538462        61.538462   
10701            72.727273         61.538462        61.538462   
10981            72.727273         61.538462        61.538462   
10416            66.666667         61.538462        61.538462   

       jaro_winkler_score  
10611           97.500000  
11095           81.547619  
10428           97.500000  
10443           97.500000  
10917           97.500000  
10551           97.500000  
10114           97.142857  
10240           97.142857  
10307           85.714286  
10427           92.777778  
10431           95.238095  
10558           95.555556  
10631           86.428571  
10557           94.285714  
10498           97.142857  
11100           94.285714  
11153           77.142857  
10596           54.304029  
10197           77.142857  
10701           66.269841  
10981           66.269841  
10416           79.682540

In [88]:
fuzzy_words_df[(fuzzy_words_df['original_value']=='bahamas') & (fuzzy_words_df['partial_ratio_score']>70)].sort_values('partial_ratio_score', ascending=False)

original_index original_value  match_index    match_value  ratio_score  \
10431               9        bahamas          914         ahamas    92.307692   
10428               9        bahamas          907       bahamasc    93.333333   
10611               9        bahamas         1201       bahamas6    93.333333   
10596               9        bahamas         1184  nassaubahamas    70.000000   
11095               9        bahamas         1912       abahamas    93.333333   
10558               9        bahamas         1125      bahamaspo    87.500000   
10089               9        bahamas          123              s    25.000000   
10550               9        bahamas         1100             ba    44.444444   
10443               9        bahamas          930       bahamas1    93.333333   
10439               9        bahamas          926             as    44.444444   
10788               9        bahamas         1426              m    25.000000   
10917               9        bahamas         1603       bahamasa    93.333333   
10374               9        bahamas          806            bah    60.000000   
11100               9        bahamas         1932          baham    83.333333   
10327               9        bahamas          718              b    25.000000   
10182               9        bahamas          355              h    25.000000   
10134               9        bahamas          238              a    25.000000   
11153               9        bahamas         2031          hamas    83.333333   
10114               9        bahamas          188         bahama    92.307692   
10551               9        bahamas         1101       bahamaas    93.333333   
10307               9        bahamas          659         bhamas    92.307692   
10240               9        bahamas          516         bahams    92.307692   
10498               9        bahamas         1018        bahaams    85.714286   
10557               9        bahamas         1124        bahanas    85.714286   
10631               9        bahamas         1228        brahmas    85.714286   
10427               9        bahamas          904         bahmas    92.307692   
11101               9        bahamas         1935            bat    40.000000   
10946               9        bahamas         1649            tba    40.000000   
10048               9        bahamas           34            bay    40.000000   
10681               9        bahamas         1300            aps    40.000000   
10566               9        bahamas         1138            bak    40.000000   
11158               9        bahamas         2036            bav    40.000000   
10640               9        bahamas         1244          massa    50.000000   
10332               9        bahamas          724          beach    50.000000   
10126               9        bahamas          222          marsh    50.000000   
10701               9        bahamas         1321         champs    61.538462   
10981               9        bahamas         1709         thomas    61.538462   
10197               9        bahamas          397         bosham    61.538462   

       partial_ratio_score  token_sort_score  token_set_score  \
10431           100.000000         92.307692        92.307692   
10428           100.000000         93.333333        93.333333   
10611           100.000000         93.333333        93.333333   
10596           100.000000         70.000000        70.000000   
11095           100.000000         93.333333        93.333333   
10558           100.000000         87.500000        87.500000   
10089           100.000000         25.000000        25.000000   
10550           100.000000         44.444444        44.444444   
10443           100.000000         93.333333        93.333333   
10439           100.000000         44.444444        44.444444   
10788           100.000000         25.000000        25.000000   
10917           100.000000         93.333333        93.333333   
10

In [89]:
fuzzy_words_df[(fuzzy_words_df['original_value']=='bahamas') & (fuzzy_words_df['token_sort_score']>65)].sort_values('token_sort_score', ascending=False)

original_index original_value  match_index    match_value  ratio_score  \
10428               9        bahamas          907       bahamasc    93.333333   
10443               9        bahamas          930       bahamas1    93.333333   
11095               9        bahamas         1912       abahamas    93.333333   
10551               9        bahamas         1101       bahamaas    93.333333   
10917               9        bahamas         1603       bahamasa    93.333333   
10611               9        bahamas         1201       bahamas6    93.333333   
10114               9        bahamas          188         bahama    92.307692   
10307               9        bahamas          659         bhamas    92.307692   
10427               9        bahamas          904         bahmas    92.307692   
10431               9        bahamas          914         ahamas    92.307692   
10240               9        bahamas          516         bahams    92.307692   
10558               9        bahamas         1125      bahamaspo    87.500000   
10631               9        bahamas         1228        brahmas    85.714286   
10557               9        bahamas         1124        bahanas    85.714286   
10498               9        bahamas         1018        bahaams    85.714286   
11100               9        bahamas         1932          baham    83.333333   
11153               9        bahamas         2031          hamas    83.333333   
10596               9        bahamas         1184  nassaubahamas    70.000000   

       partial_ratio_score  token_sort_score  token_set_score  \
10428           100.000000         93.333333        93.333333   
10443           100.000000         93.333333        93.333333   
11095           100.000000         93.333333        93.333333   
10551            92.307692         93.333333        93.333333   
10917           100.000000         93.333333        93.333333   
10611           100.000000         93.333333        93.333333   
10114           100.000000         92.307692        92.307692   
10307            90.909091         92.307692        92.307692   
10427            83.333333         92.307692        92.307692   
10431           100.000000         92.307692        92.307692   
10240            90.909091         92.307692        92.307692   
10558           100.000000         87.500000        87.500000   
10631            85.714286         85.714286        85.714286   
10557            85.714286         85.714286        85.714286   
10498            85.714286         85.714286        85.714286   
11100           100.000000         83.333333        83.333333   
11153           100.000000         83.333333        83.333333   
10596           100.000000         70.000000        70.000000   

       jaro_winkler_score  
10428           97.500000  
10443           97.500000  
11095           81.547619  
10551           97.500000  
10917           97.500000  
10611           97.500000  
10114           97.142857  
10307           85.714286  
10427           92.777778  
10431           95.238095  
10240           97.142857  
10558           95.555556  
10631           86.428571  
10557           94.285714  
10498           97.142857  
11100           94.285714  
11153           77.142857  
10596           54.304029

In [90]:
fuzzy_words_df[(fuzzy_words_df['original_value']=='bahamas') & (fuzzy_words_df['token_set_score']>65)].sort_values('token_set_score', ascending=False)

original_index original_value  match_index    match_value  ratio_score  \
10428               9        bahamas          907       bahamasc    93.333333   
10443               9        bahamas          930       bahamas1    93.333333   
11095               9        bahamas         1912       abahamas    93.333333   
10551               9        bahamas         1101       bahamaas    93.333333   
10917               9        bahamas         1603       bahamasa    93.333333   
10611               9        bahamas         1201       bahamas6    93.333333   
10114               9        bahamas          188         bahama    92.307692   
10307               9        bahamas          659         bhamas    92.307692   
10427               9        bahamas          904         bahmas    92.307692   
10431               9        bahamas          914         ahamas    92.307692   
10240               9        bahamas          516         bahams    92.307692   
10558               9        bahamas         1125      bahamaspo    87.500000   
10631               9        bahamas         1228        brahmas    85.714286   
10557               9        bahamas         1124        bahanas    85.714286   
10498               9        bahamas         1018        bahaams    85.714286   
11100               9        bahamas         1932          baham    83.333333   
11153               9        bahamas         2031          hamas    83.333333   
10596               9        bahamas         1184  nassaubahamas    70.000000   

       partial_ratio_score  token_sort_score  token_set_score  \
10428           100.000000         93.333333        93.333333   
10443           100.000000         93.333333        93.333333   
11095           100.000000         93.333333        93.333333   
10551            92.307692         93.333333        93.333333   
10917           100.000000         93.333333        93.333333   
10611           100.000000         93.333333        93.333333   
10114           100.000000         92.307692        92.307692   
10307            90.909091         92.307692        92.307692   
10427            83.333333         92.307692        92.307692   
10431           100.000000         92.307692        92.307692   
10240            90.909091         92.307692        92.307692   
10558           100.000000         87.500000        87.500000   
10631            85.714286         85.714286        85.714286   
10557            85.714286         85.714286        85.714286   
10498            85.714286         85.714286        85.714286   
11100           100.000000         83.333333        83.333333   
11153           100.000000         83.333333        83.333333   
10596           100.000000         70.000000        70.000000   

       jaro_winkler_score  
10428           97.500000  
10443           97.500000  
11095           81.547619  
10551           97.500000  
10917           97.500000  
10611           97.500000  
10114           97.142857  
10307           85.714286  
10427           92.777778  
10431           95.238095  
10240           97.142857  
10558           95.555556  
10631           86.428571  
10557           94.285714  
10498           97.142857  
11100           94.285714  
11153           77.142857  
10596           54.304029

In [91]:
fuzzy_words_df[(fuzzy_words_df['original_value']=='bahamas') & (fuzzy_words_df['jaro_winkler_score']>75)].sort_values('jaro_winkler_score', ascending=False)

original_index original_value  match_index match_value  ratio_score  \
10428               9        bahamas          907    bahamasc    93.333333   
10917               9        bahamas         1603    bahamasa    93.333333   
10611               9        bahamas         1201    bahamas6    93.333333   
10551               9        bahamas         1101    bahamaas    93.333333   
10443               9        bahamas          930    bahamas1    93.333333   
10114               9        bahamas          188      bahama    92.307692   
10498               9        bahamas         1018     bahaams    85.714286   
10240               9        bahamas          516      bahams    92.307692   
10558               9        bahamas         1125   bahamaspo    87.500000   
10431               9        bahamas          914      ahamas    92.307692   
10557               9        bahamas         1124     bahanas    85.714286   
11100               9        bahamas         1932       baham    83.333333   
10427               9        bahamas          904      bahmas    92.307692   
10374               9        bahamas          806         bah    60.000000   
10631               9        bahamas         1228     brahmas    85.714286   
10307               9        bahamas          659      bhamas    92.307692   
11095               9        bahamas         1912    abahamas    93.333333   
10550               9        bahamas         1100          ba    44.444444   
10416               9        bahamas          885      bazaar    61.538462   
10197               9        bahamas          397      bosham    61.538462   
11153               9        bahamas         2031       hamas    83.333333   

       partial_ratio_score  token_sort_score  token_set_score  \
10428           100.000000         93.333333        93.333333   
10917           100.000000         93.333333        93.333333   
10611           100.000000         93.333333        93.333333   
10551            92.307692         93.333333        93.333333   
10443           100.000000         93.333333        93.333333   
10114           100.000000         92.307692        92.307692   
10498            85.714286         85.714286        85.714286   
10240            90.909091         92.307692        92.307692   
10558           100.000000         87.500000        87.500000   
10431           100.000000         92.307692        92.307692   
10557            85.714286         85.714286        85.714286   
11100           100.000000         83.333333        83.333333   
10427            83.333333         92.307692        92.307692   
10374           100.000000         60.000000        60.000000   
10631            85.714286         85.714286        85.714286   
10307            90.909091         92.307692        92.307692   
11095           100.000000         93.333333        93.333333   
10550           100.000000         44.444444        44.444444   
10416            66.666667         61.538462        61.538462   
10197            72.727273         61.538462        61.538462   
11153           100.000000         83.333333        83.333333   

       jaro_winkler_score  
10428           97.500000  
10917           97.500000  
10611           97.500000  
10551           97.500000  
10443           97.500000  
10114           97.142857  
10498           97.142857  
10240           97.142857  
10558           95.555556  
10431           95.238095  
10557           94.285714  
11100           94.285714  
10427           92.777778  
10374           86.666667  
10631           86.428571  
10307           85.714286  
11095           81.547619  
10550           80.952381  
10416           79.682540  
10197           77.142857  
11153           77.142857

In [92]:
word_list = fuzzy_words_df[(fuzzy_words_df['original_value']=='bahamas') & (fuzzy_words_df['jaro_winkler_score']>75)].sort_values('jaro_winkler_score', ascending=False)['match_value']
for word in word_list:
    print(word)
    display(df[df['address_wordlist'].apply(lambda x: word in x)])
    print('\n')

bahamasc


node_id                                                 address name  \
570  14018521  51 Frederick Street; P.O. Box N-1136; Nassau; BahamasC  NaN   

    countries country_codes       sourceID  \
570   Bahamas           BHS  Panama Papers   

                                         valid_until note  \
570  The Panama Papers  data is current through 2015  NaN   

                                      working_address  \
570  51 frederick street po box n1136 nassau bahamasc   

                                              address_wordlist  
570  [51, frederick, street, po, box, n1136, nassau, bahamasc]



bahamasa


node_id  \
1442   240054   

                                                                              address  \
1442  Winterbotham Place Marlborough & Queen Streets PO Box CB 11343 Nassau, Bahamasa   

     name countries country_codes        sourceID  \
1442  NaN   Bahamas           BHS  Offshore Leaks   

                                          valid_until note  \
1442  The Offshore Leaks data is current through 2010  NaN   

                                                                      working_address  \
1442  winterbotham place marlborough and queen street po box cb 11343 nassau bahamasa   

                                                                                  address_wordlist  
1442  [winterbotham, place, marlborough, and, queen, street, po, box, cb, 11343, nassau, bahamasa]



bahamas6


...

node_id  \
549  14012324   

                                                                            address  \
549  2nd Floor; International Bazaar; Bay Street; P.O. Box N- 1612; Nassau; Bahamas   

    name countries country_codes       sourceID  \
549  NaN   Bahamas           BHS  Panama Papers   

                                         valid_until note  \
549  The Panama Papers  data is current through 2015  NaN   

                                                               working_address  \
549  second floor international bazaar bay street po box n 1612 nassau bahamas   

                                                                           address_wordlist  
549  [second, floor, international, bazaar, bay, street, po, box, n, 1612, nassau, bahamas]



bosham


node_id  \
199    24000200   
1887   33000077   
2180  240003759   

                                                                     address  \
199   #6 BOSHAM CLOSE, CAMPERDOWN HEIGHTS P.O. BOX SP 63801, NASSAU, BAHAMAS   
1887    #6 BOSHAM CLOSE, CAMPERDOWN HEIGHTS PO BOX SP 63801, NASSAU, BAHAMAS   
2180           NO. 6 BOSHAM CLOSE, CAMPERDOWN HEIGHTS NEW PROVIDENCE BAHAMAS   

     name countries country_codes  \
199   NaN   Bahamas           BHS   
1887  NaN   Bahamas           BHS   
2180  NaN   Bahamas           BHS   

                                                       sourceID  \
199                                               Bahamas Leaks   
1887               Paradise Papers - Bahamas corporate registry   
2180  Pandora Papers - Alemán, Cordero, Galindo & Lee (Alcogal)   

                                                  valid_until note  \
199     The Bahamas Leaks data is current through early 2016.  NaN   
1887  Bahamas corporate registry data is current through 2016  NaN   
2180                    Provider data is current through 2018  NaN   

                                                       working_address  \
199   6 bosham close camperdown heights po box sp 63801 nassau bahamas   
1887  6 bosham close camperdown heights po box sp 63801 nassau bahamas   
2180       no 6 bosham close camperdown heights new providence bahamas   

                                                                  address_wordlist  
199   [6, bosham, close, camperdown, heights, po, box, sp, 63801, nassau, bahamas]  
1887  [6, bosham, close, camperdown, heights, po, box, sp, 63801, nassau, bahamas]  
2180         [no, 6, bosham, close, camperdown, heights, new, providence, bahamas]



hamas


node_id  \
2243  240492153   

                                                                             address  \
2243  MONTAGUE STERLING CENTRE. EAST BAY STREET, NASSAU, HAMAS, SWITZERLAND, BAHAMAS   

     name countries country_codes                        sourceID  \
2243  NaN   Bahamas           BHS  Pandora Papers - Trident Trust   

                                valid_until note  \
2243  Provider data is current through 2016  NaN   

                                                                working_address  \
2243  montague sterling centre east bay street nassau hamas switzerland bahamas   

                                                                          address_wordlist  
2243  [montague, sterling, centre, east, bay, street, nassau, hamas, switzerland, bahamas]

### Nassau

In [93]:
fuzzy_words_df[(fuzzy_words_df['original_value']=='nassau') & (fuzzy_words_df['ratio_score']>60)].sort_values('ratio_score', ascending=False)

original_index original_value  match_index    match_value  ratio_score  \
9116               8         nassau          698        nasssau    92.307692   
9254               8         nassau          925        nassaub    92.307692   
9404               8         nassau         1160        nassaau    92.307692   
9460               8         nassau         1225        nassaus    92.307692   
9753               8         nassau         1594        naussau    92.307692   
9223               8         nassau          872          nasau    90.909091   
9431               8         nassau         1195          nassu    90.909091   
9356               8         nassau         1083         massau    83.333333   
9393               8         nassau         1146         nassua    83.333333   
9422               8         nassau         1186         nassao    83.333333   
9438               8         nassau         1202         nassan    83.333333   
9296               8         nassau          993      343nassau    80.000000   
9474               8         nassau         1244          massa    72.727273   
9420               8         nassau         1184  nassaubahamas    63.157895   

      partial_ratio_score  token_sort_score  token_set_score  \
9116            83.333333         92.307692        92.307692   
9254           100.000000         92.307692        92.307692   
9404            90.909091         92.307692        92.307692   
9460           100.000000         92.307692        92.307692   
9753            83.333333         92.307692        92.307692   
9223            80.000000         90.909091        90.909091   
9431            88.888889         90.909091        90.909091   
9356            90.909091         83.333333        83.333333   
9393            90.909091         83.333333        83.333333   
9422            90.909091         83.333333        83.333333   
9438            90.909091         83.333333        83.333333   
9296           100.000000         80.000000        80.000000   
9474            80.000000         72.727273        72.727273   
9420           100.000000         63.157895        63.157895   

      jaro_winkler_score  
9116           97.142857  
9254           97.142857  
9404           97.142857  
9460           97.142857  
9753           96.190476  
9223           96.111111  
9431           96.666667  
9356           88.888889  
9393           96.666667  
9422           93.333333  
9438           93.333333  
9296           88.888889  
9474           82.222222  
9420           89.230769

In [94]:
fuzzy_words_df[(fuzzy_words_df['original_value']=='nassau') & (fuzzy_words_df['partial_ratio_score']>70)].sort_values('partial_ratio_score', ascending=False)

original_index original_value  match_index    match_value  ratio_score  \
8660               8         nassau           49              n    28.571429   
9420               8         nassau         1184  nassaubahamas    63.157895   
8798               8         nassau          238              a    28.571429   
9001               8         nassau          530             ss    50.000000   
9860               8         nassau         1756             na    50.000000   
9460               8         nassau         1225        nassaus    92.307692   
9254               8         nassau          925        nassaub    92.307692   
9255               8         nassau          926             as    50.000000   
9296               8         nassau          993      343nassau    80.000000   
8713               8         nassau          123              s    28.571429   
9438               8         nassau         1202         nassan    83.333333   
9422               8         nassau         1186         nassao    83.333333   
9393               8         nassau         1146         nassua    83.333333   
9404               8         nassau         1160        nassaau    92.307692   
9356               8         nassau         1083         massau    83.333333   
9431               8         nassau         1195          nassu    90.909091   
9322               8         nassau         1034           nash    60.000000   
9753               8         nassau         1594        naussau    92.307692   
9116               8         nassau          698        nasssau    92.307692   
9223               8         nassau          872          nasau    90.909091   
9474               8         nassau         1244          massa    72.727273   
9824               8         nassau         1696           issa    60.000000   
9861               8         nassau         1758           rasa    60.000000   

      partial_ratio_score  token_sort_score  token_set_score  \
8660           100.000000         28.571429        28.571429   
9420           100.000000         63.157895        63.157895   
8798           100.000000         28.571429        28.571429   
9001           100.000000         50.000000        50.000000   
9860           100.000000         50.000000        50.000000   
9460           100.000000         92.307692        92.307692   
9254           100.000000         92.307692        92.307692   
9255           100.000000         50.000000        50.000000   
9296           100.000000         80.000000        80.000000   
8713           100.000000         28.571429        28.571429   
9438            90.909091         83.333333        83.333333   
9422            90.909091         83.333333        83.333333   
9393            90.909091         83.333333        83.333333   
9404            90.909091         92.307692        92.307692   
9356            90.909091         83.333333        83.333333   
9431            88.888889         90.909091        90.909091   
9322            85.714286         60.000000        60.000000   
9753            83.333333         92.307692        92.307692   
9116            83.333333         92.307692        92.307692   
9223            80.000000         90.909091        90.909091   
9474            80.000000         72.727273        72.727273   
9824            75.000000         60.000000        60.000000   
9861            75.000000         60.000000        60.000000   

      jaro_winkler_score  
8660           75.000000  
9420           89.230769  
8798           72.222222  
9001           77.777778  
9860           82.222222  
9460           97.142857  
9254           97.142857  
9255           77.777778  
9296           88.888889  
8713           72.222222  
9438           93.333333  
9422           93.333333  
9393           96.666667  
9404           97.142857  
9356           88.888889  
9431           96.666667  
9322           82.500000  
9753           96.190476  
9116           97.142857  
9223    

In [100]:
fuzzy_words_df[(fuzzy_words_df['original_value']=='nassau') & (fuzzy_words_df['token_sort_score']>=80)].sort_values('token_sort_score', ascending=False)

original_index original_value  match_index match_value  ratio_score  \
9116               8         nassau          698     nasssau    92.307692   
9254               8         nassau          925     nassaub    92.307692   
9404               8         nassau         1160     nassaau    92.307692   
9460               8         nassau         1225     nassaus    92.307692   
9753               8         nassau         1594     naussau    92.307692   
9223               8         nassau          872       nasau    90.909091   
9431               8         nassau         1195       nassu    90.909091   
9356               8         nassau         1083      massau    83.333333   
9393               8         nassau         1146      nassua    83.333333   
9422               8         nassau         1186      nassao    83.333333   
9438               8         nassau         1202      nassan    83.333333   
9296               8         nassau          993   343nassau    80.000000   

      partial_ratio_score  token_sort_score  token_set_score  \
9116            83.333333         92.307692        92.307692   
9254           100.000000         92.307692        92.307692   
9404            90.909091         92.307692        92.307692   
9460           100.000000         92.307692        92.307692   
9753            83.333333         92.307692        92.307692   
9223            80.000000         90.909091        90.909091   
9431            88.888889         90.909091        90.909091   
9356            90.909091         83.333333        83.333333   
9393            90.909091         83.333333        83.333333   
9422            90.909091         83.333333        83.333333   
9438            90.909091         83.333333        83.333333   
9296           100.000000         80.000000        80.000000   

      jaro_winkler_score  
9116           97.142857  
9254           97.142857  
9404           97.142857  
9460           97.142857  
9753           96.190476  
9223           96.111111  
9431           96.666667  
9356           88.888889  
9393           96.666667  
9422           93.333333  
9438           93.333333  
9296           88.888889

In [103]:
fuzzy_words_df[(fuzzy_words_df['original_value']=='nassau') & (fuzzy_words_df['token_set_score']>75)].sort_values('token_set_score', ascending=False)

original_index original_value  match_index match_value  ratio_score  \
9116               8         nassau          698     nasssau    92.307692   
9254               8         nassau          925     nassaub    92.307692   
9404               8         nassau         1160     nassaau    92.307692   
9460               8         nassau         1225     nassaus    92.307692   
9753               8         nassau         1594     naussau    92.307692   
9223               8         nassau          872       nasau    90.909091   
9431               8         nassau         1195       nassu    90.909091   
9356               8         nassau         1083      massau    83.333333   
9393               8         nassau         1146      nassua    83.333333   
9422               8         nassau         1186      nassao    83.333333   
9438               8         nassau         1202      nassan    83.333333   
9296               8         nassau          993   343nassau    80.000000   

      partial_ratio_score  token_sort_score  token_set_score  \
9116            83.333333         92.307692        92.307692   
9254           100.000000         92.307692        92.307692   
9404            90.909091         92.307692        92.307692   
9460           100.000000         92.307692        92.307692   
9753            83.333333         92.307692        92.307692   
9223            80.000000         90.909091        90.909091   
9431            88.888889         90.909091        90.909091   
9356            90.909091         83.333333        83.333333   
9393            90.909091         83.333333        83.333333   
9422            90.909091         83.333333        83.333333   
9438            90.909091         83.333333        83.333333   
9296           100.000000         80.000000        80.000000   

      jaro_winkler_score  
9116           97.142857  
9254           97.142857  
9404           97.142857  
9460           97.142857  
9753           96.190476  
9223           96.111111  
9431           96.666667  
9356           88.888889  
9393           96.666667  
9422           93.333333  
9438           93.333333  
9296           88.888889

In [107]:
fuzzy_words_df[(fuzzy_words_df['original_value']=='nassau') & (fuzzy_words_df['jaro_winkler_score']>85)].sort_values('jaro_winkler_score', ascending=False)

original_index original_value  match_index    match_value  ratio_score  \
9116               8         nassau          698        nasssau    92.307692   
9254               8         nassau          925        nassaub    92.307692   
9404               8         nassau         1160        nassaau    92.307692   
9460               8         nassau         1225        nassaus    92.307692   
9393               8         nassau         1146         nassua    83.333333   
9431               8         nassau         1195          nassu    90.909091   
9753               8         nassau         1594        naussau    92.307692   
9223               8         nassau          872          nasau    90.909091   
9422               8         nassau         1186         nassao    83.333333   
9438               8         nassau         1202         nassan    83.333333   
9420               8         nassau         1184  nassaubahamas    63.157895   
9356               8         nassau         1083         massau    83.333333   
9296               8         nassau          993      343nassau    80.000000   

      partial_ratio_score  token_sort_score  token_set_score  \
9116            83.333333         92.307692        92.307692   
9254           100.000000         92.307692        92.307692   
9404            90.909091         92.307692        92.307692   
9460           100.000000         92.307692        92.307692   
9393            90.909091         83.333333        83.333333   
9431            88.888889         90.909091        90.909091   
9753            83.333333         92.307692        92.307692   
9223            80.000000         90.909091        90.909091   
9422            90.909091         83.333333        83.333333   
9438            90.909091         83.333333        83.333333   
9420           100.000000         63.157895        63.157895   
9356            90.909091         83.333333        83.333333   
9296           100.000000         80.000000        80.000000   

      jaro_winkler_score  
9116           97.142857  
9254           97.142857  
9404           97.142857  
9460           97.142857  
9393           96.666667  
9431           96.666667  
9753           96.190476  
9223           96.111111  
9422           93.333333  
9438           93.333333  
9420           89.230769  
9356           88.888889  
9296           88.888889

In [98]:
word_list = fuzzy_words_df[(fuzzy_words_df['original_value']=='nassau') & (fuzzy_words_df['jaro_winkler_score']>85)].sort_values('jaro_winkler_score', ascending=False)['match_value']
for word in word_list:
    print(word)
    display(df[df['address_wordlist'].apply(lambda x: word in x)])
    print('\n')

nassaau


node_id  \
966  14078961   

                                                                      address  \
966  Suite 102; Saffrey Square; Bay Street and Bank Lane Nassaau; The Bahamas   

    name countries country_codes       sourceID  \
966  NaN   Bahamas           BHS  Panama Papers   

                                         valid_until note  \
966  The Panama Papers  data is current through 2015  NaN   

                                                           working_address  \
966  suite 102 saffrey square bay street and bank lane nassaau the bahamas   

                                                                       address_wordlist  
966  [suite, 102, saffrey, square, bay, street, and, bank, lane, nassaau, the, bahamas]



nasssau


node_id  \
420  24000421   

                                                                      address  \
420  3RD FLOOR, GEORGE HOUSE, GEORGE STREET, P.O. BOX N-8159 NASSSAU, BAHAMAS   

    name countries country_codes       sourceID  \
420  NaN   Bahamas           BHS  Bahamas Leaks   

                                               valid_until note  \
420  The Bahamas Leaks data is current through early 2016.  NaN   

                                                         working_address  \
420  third floor george house george street po box n8159 nasssau bahamas   

                                                                    address_wordlist  
420  [third, floor, george, house, george, street, po, box, n8159, nasssau, bahamas]



nassaub


...

node_id  \
287   24000288   
349   24000350   
355   24000356   
454   24000455   
788   14047561   
1668  81056994   
1688  81062872   
1743  81081828   
1756  81084980   

                                                                              address  \
287                                                P.O. BOX SS 19051, NASSAU, BAHAMAS   
349           35 DURHAM STREET OFF MONTROSE AVENUE, P.O. BOX SS 6171, NASSAU, BAHAMAS   
355                             #8 GROSVENOR CLOSE, P.O. BOX SS 6502, NASSAU, BAHAMAS   
454                       RUBY HOUSE 37 WULFF ROAD, P.O. BOX SS 5342, NASSAU, BAHAMAS   
788   L&L CORPORATE & LEGAL SERVICES, EAST BAY STREET P O BOX SS 19812 NASSAU BAHAMAS   
1668                                              P.O. Box ss - 5539; Nassau; Bahamas   
1688                                Eastern Road; Sea Spay; Nassau ss+ 19520; Bahamas   
1743     Centre of Commerce; 2nd Floor; 1 Bay Street; PO Box SS 6289; Nassau; Bahamas   
1756              PO Box SS 5985; 8 Brace Ridge Road; Nassau; New Providence; Bahamas   

                    name countries country_codes                   sourceID  \
287                  NaN   Bahamas           BHS              Bahamas Leaks   
349                  NaN   Bahamas           BHS              Bahamas Leaks   
355                  NaN   Bahamas           BHS              Bahamas Leaks   
454                  NaN   Bahamas           BHS              Bahamas Leaks   
788                  NaN   Bahamas           BHS              Panama Papers   
1668  P.O. Box ss - 5539   Bahamas           BHS  Paradise Papers - Appleby   
1688        Eastern Road   Bahamas           BHS  Paradise Papers - Appleby   
1743  Centre of Commerce   Bahamas           BHS  Paradise Papers - Appleby   
1756      PO Box SS 5985   Bahamas           BHS  Paradise Papers - Appleby   

                                                valid_until note  \
287   The Bahamas Leaks data is current through early 2016.  NaN   
349   The Bahamas Leaks data is current through early 2016.  NaN   
355   The Bahamas Leaks data is current through early 2016.  NaN   
454   The Bahamas Leaks data is current through early 2016.  NaN   
788         The Panama Papers  data is current through 2015  NaN   
1668                   Appleby data is current through 2014  NaN   
1688                   Appleby data is current through 2014  NaN   
1743                   Appleby data is current through 2014  NaN   
1756                   Appleby data is current through 2014  NaN   

                                                                        working_address  \
287                                                      po box ss 19051 nassau bahamas   
349                  35 durham street off montrose avenue po box ss 6171 nassau bahamas   
355                                     8 grosvenor close po box ss 6502 nassau bahamas   
454                              ruby house 37 wulff road po box ss 5342 nassau bahamas   
788   landl corporate and legal services east bay street po box ss 19812 nassau bahamas   
1668                                                      po box ss 5539 nassau bahamas   
1688                                      eastern road sea spay nassau ss 19520 bahamas   
1743         centre of commerce second floor 1 bay street po box ss 6289 nassau bahamas   
1756                    po box ss 5985 8 brace ridge road nassau new providence bahamas   

                                                                                      address_wordlist  
287                                                              [po, box, ss, 19051, nassau, bahamas]  
349                    [35, durham, street, off, montrose, avenue, po, box, ss, 6171, nassau, bahamas]  
355                                          [8, grosvenor, close, po, box, ss, 6502, nassau, bahamas]  
454                                 [ruby, house, 37, wulff, road, po, box, ss, 5342, nassau, bahamas]  
788   [landl, corporate, an



as


node_id  \
585   14028452   
773   14043947   
778   14045483   
1193  14087509   
1580  81028921   

                                                                                                                                       address  \
585                                                   as trustees of the Wave Trust - Charlotte House; Charlotte Street; P.O. Box N-65; Nassau   
773                                                                                         Guernsey as trustees of the Archon Trust (Bahamas)   
778   Internally Registered as Holder: DITEL; INC.; registered address: Saffrey Square; Suite 205; Bank Lane; P.O. Box # 8188; Nassau; Bahamas   
1193      (THE PRIVATE CORPORATION LIMITED AS TRUSTEE OF THE MANTOLINE TRUST)Charlotte House; Charlotte Street; P.O. Box N-65; Nassau; Bahamas   
1580                                   As Trustee of The Tuleu Family Settlement; 3 Floor; Scotiabank Building; Rawson Square; Nassau; Bahamas   

                                           name countries country_codes  \
585                                         NaN   Bahamas           BHS   
773                                         NaN   Bahamas           BHS   
778                                         NaN   Bahamas           BHS   
1193                                        NaN   Bahamas           BHS   
1580  As Trustee of The Tuleu Family Settlement   Bahamas           BHS   

                       sourceID  \
585               Panama Papers   
773               Panama Papers   
778               Panama Papers   
1193              Panama Papers   
1580  Paradise Papers - Appleby   

                                          valid_until note  \
585   The Panama Papers  data is current through 2015  NaN   
773   The Panama Papers  data is current through 2015  NaN   
778   The Panama Papers  data is current through 2015  NaN   
1193  The Panama Papers  data is current through 2015  NaN   
1580             Appleby data is current through 2014  NaN   

                                                                                                                  working_address  \
585                                           as trustees of the wavenue trust charlotte house charlotte street po box n65 nassau   
773                                                                              guernsey as trustees of the archon trust bahamas   
778    internally registered as holder ditel inc registered address saffrey square suite 205 bank lane po box 8188 nassau bahamas   
1193  the private corporation limited as trustee of the mantoline trustcharlotte house charlotte street po box n65 nassau bahamas   
1580                           as trustee of the tuleu family settlement 3 floor scotiabank building rawson square nassau bahamas   

                                                                                                                                    address_wordlist  
585                                               [as, trustees, of, the, wavenue, trust, charlotte, house, charlotte, street, po, box, n65, nassau]  
773                                                                                        [guernsey, as, trustees, of, the, archon, trust, bahamas]  
778   [internally, registered, as, holder, ditel, inc, registered, address, saffrey, square, suite, 205, bank, lane, po, box, 8188, nassau, bahamas]  
1193  [the, private, corporation, limited, as, trustee, of, the, mantoline, trustcharlotte, house, charlotte, street, po, box, n65, nassau, bahamas]  
1580                              [as, trustee, of, the, tuleu, family, settlement, 3, floor, scotiabank, building, rawson, square, nassau, bahamas]



n


node_id  \
9      24000010   
23     24000024   
40     24000041   
78     24000079   
82     24000083   
...         ...   
2245  240492204   
2247  240492217   
2251  240492292   
2253  240492525   
2257  240491892   

                                                                                                                                          address  \
9                                                                      GROUND FLOOR, GOODMAN'S BAY CORPORATE CE, P.O. BOX N 3933, NASSAU, BAHAMAS   
23                                                                                                                P.O. BOX N 529, NASSAU, BAHAMAS   
40                                                                                          ONE MONTAGUE PLACE, P.O. BOX. N-4906, NASSAU, BAHAMAS   
78                                                                           #10 PETRONA HOUSE, FOWLER ST. EAST, P.O. BOX N 1375, NASSAU, BAHAMAS   
82                                                                                                               P.O. BOX N 4818, NASSAU, BAHAMAS   
...                                                                                                                                           ...   
2245       J.P. MORGAN TRUST COMPANY (BAHAMAS) LIMITED, 2ND FLOOR BAHAMAS FINANCIAL CENTRE, SHIRLEY AND CHARLOTTE STREET, NASSAU, N-4899, BAHAMAS   
2247                                                                         J.P. MORGAN TRUST COMPANY (BAHAMAS) LIMITED, NASSAU, N-4899, BAHAMAS   
2251                                     J.P. MORGAN TRUST COMPANY (BAHAMAS) LIMITED, 2ND FLOOR BAHAMAS FINANCIAL CENTRE, NASSAU, N-4899, BAHAMAS   
2253                                                                      J.P.MORGAN TRUST COMPANY (BAHAMAS) LIMITED, NASSAU, N-4899, ZH, BAHAMAS   
2257  C/OJ.P. MORGAN TRUST COMPANY (BAHAMAS) LIMITED, BAHAMAS FINANCIAL CENTER 2NTL FLOOR, SHIRLEY AND CHARLOTTE STREETS, NASSAU, N-4899, BAHAMAS   

     name countries country_codes                        sourceID  \
9     NaN   Bahamas           BHS                   Bahamas Leaks   
23    NaN   Bahamas           BHS                   Bahamas Leaks   
40    NaN   Bahamas           BHS                   Bahamas Leaks   
78    NaN   Bahamas           BHS                   Bahamas Leaks   
82    NaN   Bahamas           BHS                   Bahamas Leaks   
...   ...       ...           ...                             ...   
2245  NaN   Bahamas           BHS  Pandora Papers - Trident Trust   
2247  NaN   Bahamas           BHS  Pandora Papers - Trident Trust   
2251  NaN   Bahamas           BHS  Pandora Papers - Trident Trust   
2253  NaN   Bahamas           BHS  Pandora Papers - Trident Trust   
2257  NaN   Bahamas           BHS  Pandora Papers - Trident Trust   

                                                valid_until note  \
9     The Bahamas Leaks data is current through early 2016.  NaN   
23    The Bahamas Leaks data is current through early 2016.  NaN   
40    The Bahamas Leaks data is current through early 2016.  NaN   
78    The Bahamas Leaks data is current through early 2016.  NaN   
82    The Bahamas Leaks data is current through early 2016.  NaN   
...                                                     ...  ...   
2245                  Provider data is current through 2017  NaN   
2247                  Provider data is current through 2016  NaN   
2251                  Provider data is current through 2017  NaN   
2253                  Provider data is current through 2016  NaN   
2257                  Provider data is current through 2017  NaN   

                                                                                                                        working_address  \
9                                                                   ground floor goodmans bay corporate ce po box n 3933 nassau bahamas   
23                                                                

In [108]:
test_list = ['suite', 'e', '2', 'union', 'court', 'building', 'elizabeth', 'avenue', 'and', 'shirly', 'street', 'nassan', 'the', 'bahamas']

In [109]:
test_list[:-1]

['suite',
 'e',
 '2',
 'union',
 'court',
 'building',
 'elizabeth',
 'avenue',
 'and',
 'shirly',
 'street',
 'nassan',
 'the']

In [113]:
df[df['address_wordlist'].apply(lambda x: 'bah' in x)]

node_id  \
495  24000496   
760  14043538   

                                                                         address  \
495  SHIRLEY & CHARLOTTE STS BAH. FIN. CENTRE, P.O. BOX SS-6373, NASSAU, BAHAMAS   
760                  GOODMAN S  BAY CORPORATE CENTER WEST BAY STREET NASSAU  BAH   

    name countries country_codes       sourceID  \
495  NaN   Bahamas           BHS  Bahamas Leaks   
760  NaN   Bahamas           BHS  Panama Papers   

                                               valid_until note  \
495  The Bahamas Leaks data is current through early 2016.  NaN   
760        The Panama Papers  data is current through 2015  NaN   

                                                              working_address  \
495  shirley and charlotte street bah fin centre po box ss6373 nassau bahamas   
760                 goodman s bay corporate center west bay street nassau bah   

                                                                          address_wordlist  
495  [shirley, and, charlotte, street, bah, fin, centre, po, box, ss6373, nassau, bahamas]  
760                   [goodman, s, bay, corporate, center, west, bay, street, nassau, bah]

In [120]:
pd.set_option('display.max_rows', 70)

In [121]:
df['address_wordlist'].apply(lambda x: x[-1]).value_counts()

bahamas          2040
bahama             66
nassau             54
freeport            5
street              5
bahmas              5
bahams              5
i                   4
providence          3
nassaubahamas       3
centre              3
lane                3
343                 3
kln                 2
abaco               2
bahamas1            2
bhs                 2
kelty               2
2423527291          2
prat                1
lotmore             1
bay                 1
esquare             1
massa               1
charlotte           1
n3023               1
co                  1
cb11323             1
isle                1
bahamasa            1
sandyport           1
abahamas            1
bahamas6            1
cay                 1
44311               1
cb1911              1
building            1
99001               1
23                  1
eleuthera           1
49539               1
bahamaas            1
12062001            1
square              1
ss5539              1
n10700    

In [124]:
df.loc[df['working_address'].str.contains('ba?h\w+s\w?\d?$'), 'address_wordlist'].apply(lambda x: x[-1]).value_counts()

bahamas          1954
bahams              5
bahmas              5
nassaubahamas       3
bahamas1            2
bhamas              1
bahamasc            1
bahaams             1
bahamaas            1
bahanas             1
bahamas6            1
bahamasa            1
abahamas            1
Name: address_wordlist, dtype: int64

In [ ]:
def get_set_ratio(row, str_to_match):
    name = row['address']
    return fuzz.token_set_ratio(name, str_to_match)

def get_sort_ratio(row, str_to_match):
    name = row['address']
    return fuzz.token_sort_ratio(name, str_to_match)

In [ ]:
u_addresses = df['working_address'].unique()
for u_address in u_addresses:
    df['score'] = df.apply(get_sort_ratio, args=(u_address,), axis=1)
    match_df = sm_df[sm_df['score']>80].sort_values('score', ascending=False)
    if match_df.shape[0]>1:
        print(u_address)
        display(match_df)
        print('\n')

# Fuzzy Resources

- [Fuzzing matching in pandas with fuzzywuzzy](https://jonathansoma.com/lede/algorithms-2017/classes/fuzziness-matplotlib/fuzzing-matching-in-pandas-with-fuzzywuzzy/)
- [Best Libraries for Fuzzy Matching In Python](https://medium.com/codex/best-libraries-for-fuzzy-matching-in-python-cbb3e0ef87dd)
- [Fuzzy String Matching](https://towardsdatascience.com/fuzzy-string-matching-in-python-68f240d910fe)
- [Fuzzy String Comparison](https://stackoverflow.com/a/28467760)
- [How to do Fuzzy Matching on Pandas Dataframe Column Using Python?](https://www.geeksforgeeks.org/how-to-do-fuzzy-matching-on-pandas-dataframe-column-using-python/)
- []()